In [16]:
!df -h
!du -sh *

Filesystem      Size  Used Avail Use% Mounted on
overlay          30G   14G   17G  45% /
tmpfs            64M     0   64M   0% /dev
tmpfs           3.9G     0  3.9G   0% /sys/fs/cgroup
/dev/nvme1n1     98G   98G  202M 100% /home/jovyan
/dev/nvme0n1p1   30G   14G   17G  45% /etc/hosts
shm              64M  8.0K   64M   1% /dev/shm
tmpfs           7.0G  4.0K  7.0G   1% /run/secrets/tokens
tmpfs           3.9G     0  3.9G   0% /proc/acpi
tmpfs           3.9G     0  3.9G   0% /sys/firmware
128K	NMC.ipynb
236K	NMC_MD.ipynb
80K	Untitled.ipynb
8.0K	extracted_data_advanced.csv
44K	extracted_data_final.csv
96K	extracted_data_final_with_filenames-Copy1.csv
96K	extracted_data_final_with_filenames.csv
18G	output
684K	pressure.ipynb
308K	structure.ipynb


In [1]:
x=(3300*2+800+1500)/17
x+1150

1673.5294117647059

In [4]:
y=6315/6
y+1353.5

2406.0

In [15]:
import jupyter-resource-usage
help(jupyter-resource-usage)

SyntaxError: invalid syntax (1677304134.py, line 1)

In [ ]:
#!/usr/bin/env python3
"""
Trajectoryファイルの整理と、それに基づく界面構築・最適化を自動化するプログラム。

[最終版: 構築→トリミング→最適化]
- 破損したファイルなど、特定ファイルの処理でエラーが発生しても全体が停止せず、
  次のファイルの処理を続行します。

STEP 1: Trajectoryファイルの分析とクリーンアップ
  - 0KBの.trajファイル（失敗した計算）を削除
  - 正常な.trajファイルを初期/最終構造の.xyzに変換し、元ファイルを削除

STEP 2: 界面の構築、トリミング、最適化
  - STEP 1で生成された最終構造(.xyz)を用いて界面を構築
  - 構築した界面を指定の厚み（25Å/スラブ）にトリミング
  - トリミング後の構造に対してMatlantisで原子位置を最適化
"""
import os
import itertools
import numpy as np
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display, clear_output
import nglview as nv

# ASE
from ase.io import read, write
from ase.build import stack,cut

# Matlantis / PFP
from matlantis_features.atoms import MatlantisAtoms
from matlantis_features.features.common.opt import FireLBFGSASEOptFeature
from matlantis_features.utils.calculators import pfp_estimator_fn
from pfp_api_client.pfp.estimator import EstimatorCalcMode

# --- STEP 1: Trajectoryファイルの整理クラス ---
class TrajectoryCleanup:
    """
    .trajファイルを分析し、.xyzに変換してクリーンアップするクラス。
    """
    def __init__(self, directory):
        self.directory = Path(directory)
        print("--- STEP 1: Trajectoryファイルの分析と整理を開始 ---")
        if not self.directory.is_dir():
            print(f"🛑 エラー: ディレクトリ '{self.directory}' が見つかりません。")
            self.files_to_process = []
        else:
            self.files_to_process = list(self.directory.glob("*.traj"))
            print(f"🔍 '{self.directory}' から {len(self.files_to_process)} 個の .traj ファイルを発見。")

    def process_file(self, traj_path):
        """
        1つの.trajファイルを処理する。
        [修正] この関数全体をtry...exceptで囲み、エラー耐性を向上。
        """
        try:
            # ファイルサイズを確認
            file_size = traj_path.stat().st_size
            if file_size == 0:
                print(f"  -> 🗑️  0KBファイルを削除します: {traj_path.name}")
                os.remove(traj_path)
                return

            # 正常なファイルは変換
            base_name = traj_path.stem
            initial_xyz_path = self.directory / f"{base_name}_initial.xyz"
            final_xyz_path = self.directory / f"{base_name}_final.xyz"

            print(f"  -> 🔄 変換中: {traj_path.name}")

            # Trajectoryから初期構造と最終構造を読み込み
            atoms_list = read(traj_path, index=":")
            if not atoms_list:
                print(f"  -> ❌ 読み込みエラー（空のTrajectory）、ファイルを削除します: {traj_path.name}")
                os.remove(traj_path)
                return

            # .xyzファイルとして保存
            write(str(initial_xyz_path), atoms_list[0])
            write(str(final_xyz_path), atoms_list[-1])

            # 元の.trajファイルを削除
            os.remove(traj_path)
            print(f"  -> ✔️ 変換成功、元のファイルを削除しました: {traj_path.name}")

        except Exception as e:
            # 予期せぬエラーが発生した場合、ログを出力して処理を続行
            print(f"  -> ❌ 重大なエラーが発生したため、このファイルはスキップします ({traj_path.name}): {e}")

    def run(self):
        """全ての対象ファイルに対して処理を実行する。"""
        if not self.files_to_process:
            print(" -> 処理対象の.trajファイルがないため、スキップします。")
            return
        for traj_file in self.files_to_process:
            self.process_file(traj_file)
        print("\n✨ STEP 1 完了。")


# --- Matlantis最適化関数 (共通) ---
def run_matlantis_optimization(atoms, trajectory_path, fmax=0.05, name="structure"):
    print(f"  -> Matlantis最適化開始 ({name}, fmax = {fmax}) ...")
    matlantis_atoms = MatlantisAtoms(atoms)
    estimator_function = pfp_estimator_fn(
        model_version='v7.0.0', calc_mode=EstimatorCalcMode.CRYSTAL_PLUS_D3
    )
    position_optimizer = FireLBFGSASEOptFeature(
        estimator_fn=estimator_function, filter=False, trajectory=str(trajectory_path),
        n_run=5000, fmax=fmax, show_progress_bar=True
    )
    try:
        result = position_optimizer(matlantis_atoms)
        optimized_atoms = result.atoms.ase_atoms
        final_energy = result.output.energy_log[-1]
        print(f"  -> ✔️ 最適化完了！ エネルギー: {final_energy:.3f} eV")
        # .xyzファイルでの保存はここで継続
        xyz_save_path = Path(trajectory_path).with_suffix('.xyz')
        print(f"  -> 💾 最適化後の中間構造を保存します: {xyz_save_path.name}")
        write(str(xyz_save_path), optimized_atoms)
        return optimized_atoms  # ★変更点: 最適化後の構造オブジェクトを返す
    except Exception as e:
        print(f"  -> ❌ 最適化中にエラーが発生: {e}")
        return None  # ★変更点: 失敗した場合はNoneを返す

# --- STEP 2: 界面の構築・トリミング・最適化クラス ---
class InterfaceOptimizer:
    def __init__(self, surfaces_dir, interfaces_dir):
        self.surfaces_dir = Path(surfaces_dir)
        self.interfaces_dir = Path(interfaces_dir)
        self.interfaces_dir.mkdir(parents=True, exist_ok=True)
        print(f"\n--- STEP 2: 界面 構築・トリミング・最適化プロセスを開始 ---")
        print(f"✅ 最終構造の保存先: '{self.interfaces_dir}'")

    def find_and_categorize_surfaces(self):
        print(f"🔍 '{self.surfaces_dir}' から最適化済み表面 (*_final.xyz) を分類中...")
        if not self.surfaces_dir.is_dir():
            print(f"🛑 エラー: 表面ディレクトリ '{self.surfaces_dir}' が見つかりません。")
            return [], []
        all_surfaces = list(self.surfaces_dir.glob("*_final.xyz"))
        al_surfaces = [p for p in all_surfaces if "Al" in p.stem]
        cathode_surfaces = [p for p in all_surfaces if "NMC" in p.stem]
        print(f"  - Al表面: {len(al_surfaces)}個,  正極材表面: {len(cathode_surfaces)}個")
        return al_surfaces, cathode_surfaces

    def build_interface(self, slab1, slab2, target_xy=(12.0, 12.0), separation=2.0):
        print(f"    1. 界面を構築中...")
        # (以前のbuild_and_cut_interfaceから原子削除とスタック部分を流用)
        position1 = slab1.get_positions()
        position2 = slab2.get_positions()
        indices_to_delete1 = np.where((position1[:, 0] >= target_xy[0]) | (position1[:, 1] >= target_xy[1]))[0]
        indices_to_delete2 = np.where((position2[:, 0] >= target_xy[0]) | (position2[:, 1] >= target_xy[1]))[0]
        cut_slab1, cut_slab2 = slab1.copy(), slab2.copy()
        if len(indices_to_delete1) > 0: del cut_slab1[indices_to_delete1]
        if len(indices_to_delete2) > 0: del cut_slab2[indices_to_delete2]
        z1_max = cut_slab1.positions[:, 2].max()
        z2_min = cut_slab2.positions[:, 2].min()
        cut_slab2.positions[:, 2] += z1_max - z2_min + separation
        interface = cut_slab1 + cut_slab2
        interface.cell = [12, 12, 53]
        return interface

    def trim_interface(self, atoms, thickness_per_slab=25.0):
        print(f"    2. 界面をトリミング中 (各スラブ厚み: {thickness_per_slab} Å)...")
        # 界面のZ座標を推定
        al_indices = [a.index for a in atoms if a.symbol == 'Al']
        nmc_indices = [a.index for a in atoms if a.symbol not in ['Al']]
        if not al_indices or not nmc_indices: return None
        al_pos = atoms.get_positions()[al_indices]
        nmc_pos = atoms.get_positions()[nmc_indices]
        if np.mean(al_pos[:, 2]) < np.mean(nmc_pos[:, 2]):
            interface_z = (al_pos[:, 2].max() + nmc_pos[:, 2].min()) / 2
        else:
            interface_z = (nmc_pos[:, 2].max() + al_pos[:, 2].min()) / 2

        # 削除する原子を特定
        indices_to_delete = []
        is_al_bottom = np.mean(al_pos[:, 2]) < np.mean(nmc_pos[:, 2])
        for atom in atoms:
            if is_al_bottom:
                if atom.symbol == 'Al' and atom.z < interface_z - thickness_per_slab:
                    indices_to_delete.append(atom.index)
                elif atom.symbol != 'Al' and atom.z > interface_z + thickness_per_slab:
                    indices_to_delete.append(atom.index)
            else:
                if atom.symbol != 'Al' and atom.z < interface_z - thickness_per_slab:
                    indices_to_delete.append(atom.index)
                elif atom.symbol == 'Al' and atom.z > interface_z + thickness_per_slab:
                    indices_to_delete.append(atom.index)

        print(f"       -> {len(indices_to_delete)} 個の原子を削除します。")
        if len(indices_to_delete) > 0:
            del atoms[indices_to_delete]
        return atoms

    def create_trim_and_optimize(self, slab1_path, slab2_path):
        slab1_name = slab1_path.stem.replace('_pos_opt_final', '').replace('_cell_opt_final', '')
        slab2_name = slab2_path.stem.replace('_pos_opt_final', '').replace('_cell_opt_final', '')
        interface_name = f"Interface_{slab1_name}_on_{slab2_name}_trimmed"
        output_path = self.interfaces_dir / f"{interface_name}.cif"

        if output_path.exists():
            print(f"✅ 既存ファイルあり、スキップ: {output_path.name}")
            return

        print(f"\n--- 処理開始: {slab1_name} + {slab2_name} ---")
        try:
            # STEP 1: 構築
            slab1, slab2 = read(slab1_path), read(slab2_path)
            interface_built = self.build_interface(slab1, slab2)

            # STEP 2: トリミング
            interface_trimmed = self.trim_interface(interface_built, thickness_per_slab=25.0)
            if interface_trimmed is None:
                print("   -> ❌ トリミングに失敗したため、スキップします。")
                return

            # STEP 3: 最適化
            print(f"    3. トリミング後の構造を最適化中...")
            traj_path = self.interfaces_dir / f"{interface_name}.traj"
            optimized_structure = run_matlantis_optimization(interface_trimmed, traj_path, fmax=0.05, name=interface_name)

            # STEP 4: 保存
            if optimized_structure:
                optimized_structure.center(vacuum=15.0, axis=2)
                optimized_structure.pbc = (True, True, False)
                print(f"    4. 最終構造をファイルに保存中 -> {output_path.name}")
                write(str(output_path), optimized_structure)
            else:
                print("   -> ❌ 最適化に失敗したため、ファイルは保存されません。")

        except Exception as e:
            print(f"  -> ❌ この界面の作成中に重大なエラーが発生: {e}")

    def run(self):
        al_slabs, cathode_slabs = self.find_and_categorize_surfaces()
        if not al_slabs or not cathode_slabs:
            print(" -> Al表面または正極材表面のファイルが不足しているため、スキップします。")
            return

        print("\n--- Al/NMC界面の組み合わせ処理を開始 ---")
        for al_path in al_slabs:
            for cathode_path in cathode_slabs:
                self.create_trim_and_optimize(al_path, cathode_path)

        print("\n✨ STEP 2 全ての処理が完了。")


def show_results_viewer(results_dir):
    print(f"\n--- 🔬 結果ビューワー: '{results_dir}' ---")
    files = sorted(list(Path(results_dir).glob("*.cif")))
    if not files:
        print("表示できる構造がありません。")
        return
    options = {"-- 選択してください --": None}
    options.update({p.name: p for p in files})
    widgets.interact(
        lambda file_path: display(nv.show_ase(read(file_path)) if file_path else None),
        file_path=widgets.Dropdown(options=options, description='Interface:', layout={'width': 'max-content'})
    )

# --- メイン実行部 ---
if __name__ == "__main__":
    # ===== 設定項目 =====
    # 1. Trajectoryファイルが保存されているディレクトリ
    OPTIMIZATION_OUTPUT_DIR = "/home/jovyan/Kaori/MD/LiB_2/structure/output"

    # 2. 最終的な界面構造の保存先
    INTERFACES_DIR = "/home/jovyan/Kaori/MD/LiB_2/structure/output/final_interfaces2"
    # ====================

    clear_output(wait=True)

    # STEP 1: メモリ管理 (traj -> xyz変換 & 削除)
    cleanup = TrajectoryCleanup(OPTIMIZATION_OUTPUT_DIR)
    cleanup.run()

    # STEP 2: 界面の構築、トリミング、最適化を一括実行
    interface_opt = InterfaceOptimizer(surfaces_dir=OPTIMIZATION_OUTPUT_DIR, interfaces_dir=INTERFACES_DIR)
    interface_opt.run()

    print("\n\n✨ 全てのワークフローが完了しました。 ✨")

    # 結果の表示
    show_results_viewer(INTERFACES_DIR)

--- STEP 1: Trajectoryファイルの分析と整理を開始 ---
🔍 '/home/jovyan/Kaori/MD/LiB_2/structure/output' から 0 個の .traj ファイルを発見。
 -> 処理対象の.trajファイルがないため、スキップします。

--- STEP 2: 界面 構築・トリミング・最適化プロセスを開始 ---
✅ 最終構造の保存先: '/home/jovyan/Kaori/MD/LiB_2/structure/output/final_interfaces'
🔍 '/home/jovyan/Kaori/MD/LiB_2/structure/output' から最適化済み表面 (*_final.xyz) を分類中...
  - Al表面: 11個,  正極材表面: 86個

--- Al/NMC界面の組み合わせ処理を開始 ---

--- 処理開始: Al_110_(1, 1, 0) + NMC111_selective_Co_removal_110_(1, 1, 0) ---
    1. 界面を構築中...
    2. 界面をトリミング中 (各スラブ厚み: 25.0 Å)...
       -> 9 個の原子を削除します。
    3. トリミング後の構造を最適化中...
  -> Matlantis最適化開始 (Interface_Al_110_(1, 1, 0)_on_NMC111_selective_Co_removal_110_(1, 1, 0)_trimmed, fmax = 0.05) ...


  0%|          | 0/5001 [00:00<?, ?it/s]

  -> ✔️ 最適化完了！ エネルギー: -2444.426 eV
  -> 💾 最適化後の中間構造を保存します: Interface_Al_110_(1, 1, 0)_on_NMC111_selective_Co_removal_110_(1, 1, 0)_trimmed.xyz
    4. 最終構造をファイルに保存中 -> Interface_Al_110_(1, 1, 0)_on_NMC111_selective_Co_removal_110_(1, 1, 0)_trimmed.cif

--- 処理開始: Al_110_(1, 1, 0) + NMC111_Co_MntoNi_40_110_(1, 1, 0) ---
    1. 界面を構築中...
    2. 界面をトリミング中 (各スラブ厚み: 25.0 Å)...
       -> 9 個の原子を削除します。
    3. トリミング後の構造を最適化中...
  -> Matlantis最適化開始 (Interface_Al_110_(1, 1, 0)_on_NMC111_Co_MntoNi_40_110_(1, 1, 0)_trimmed, fmax = 0.05) ...


  0%|          | 0/5001 [00:00<?, ?it/s]

  -> ✔️ 最適化完了！ エネルギー: -2451.518 eV
  -> 💾 最適化後の中間構造を保存します: Interface_Al_110_(1, 1, 0)_on_NMC111_Co_MntoNi_40_110_(1, 1, 0)_trimmed.xyz
    4. 最終構造をファイルに保存中 -> Interface_Al_110_(1, 1, 0)_on_NMC111_Co_MntoNi_40_110_(1, 1, 0)_trimmed.cif

--- 処理開始: Al_110_(1, 1, 0) + NMC111_Co_MntoNi_40_012_(0, 1, 2) ---
    1. 界面を構築中...
    2. 界面をトリミング中 (各スラブ厚み: 25.0 Å)...
       -> 0 個の原子を削除します。
    3. トリミング後の構造を最適化中...
  -> Matlantis最適化開始 (Interface_Al_110_(1, 1, 0)_on_NMC111_Co_MntoNi_40_012_(0, 1, 2)_trimmed, fmax = 0.05) ...


  0%|          | 0/5001 [00:00<?, ?it/s]

  -> ✔️ 最適化完了！ エネルギー: -1556.047 eV
  -> 💾 最適化後の中間構造を保存します: Interface_Al_110_(1, 1, 0)_on_NMC111_Co_MntoNi_40_012_(0, 1, 2)_trimmed.xyz
    4. 最終構造をファイルに保存中 -> Interface_Al_110_(1, 1, 0)_on_NMC111_Co_MntoNi_40_012_(0, 1, 2)_trimmed.cif

--- 処理開始: Al_110_(1, 1, 0) + NMC111_pristine_010_(0, 1, 0) ---
    1. 界面を構築中...
    2. 界面をトリミング中 (各スラブ厚み: 25.0 Å)...
       -> 328 個の原子を削除します。
    3. トリミング後の構造を最適化中...
  -> Matlantis最適化開始 (Interface_Al_110_(1, 1, 0)_on_NMC111_pristine_010_(0, 1, 0)_trimmed, fmax = 0.05) ...


  0%|          | 0/5001 [00:00<?, ?it/s]

In [1]:
#!/usr/bin/env python3
"""
Trajectoryファイルの整理と、それに基づく界面構築・最適化を自動化するプログラム。

[追加実行版]
- AlF3とAl2O3のみをAl側表面として扱い、追加で界面構築と最適化を実行します。
- STEP1のTrajectoryファイル整理はスキップします。
"""
import os
import itertools
import numpy as np
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display, clear_output
import nglview as nv

# ASE
from ase.io import read, write
from ase.build import stack,cut

# Matlantis / PFP
from matlantis_features.atoms import MatlantisAtoms
from matlantis_features.features.common.opt import FireLBFGSASEOptFeature
from matlantis_features.utils.calculators import pfp_estimator_fn
from pfp_api_client.pfp.estimator import EstimatorCalcMode

# --- STEP 1: Trajectoryファイルの整理クラス (今回は使用しない) ---
class TrajectoryCleanup:
    def __init__(self, directory):
        print("--- STEP 1: Trajectoryファイルの分析と整理 (スキップ) ---")
    def run(self):
        pass

# --- Matlantis最適化関数 (共通) ---

def run_matlantis_optimization(atoms, trajectory_path, fmax=0.05, name="structure"):
    print(f"  -> Matlantis最適化開始 ({name}, fmax = {fmax}) ...")
    matlantis_atoms = MatlantisAtoms(atoms)
    estimator_function = pfp_estimator_fn(
        model_version='v7.0.0', calc_mode=EstimatorCalcMode.CRYSTAL_PLUS_D3
    )
    position_optimizer = FireLBFGSASEOptFeature(
        estimator_fn=estimator_function, filter=False, trajectory=str(trajectory_path),
        n_run=5000, fmax=fmax, show_progress_bar=True
    )
    try:
        result = position_optimizer(matlantis_atoms)
        optimized_atoms = result.atoms.ase_atoms
        final_energy = result.output.energy_log[-1]
        print(f"  -> ✔️ 最適化完了！ エネルギー: {final_energy:.3f} eV")
        xyz_save_path = Path(trajectory_path).with_suffix('.xyz')
        print(f"  -> 💾 最適化後の中間構造を保存します: {xyz_save_path.name}")
        write(str(xyz_save_path), optimized_atoms)
        return optimized_atoms
    except Exception as e:
        print(f"  -> ❌ 最適化中にエラーが発生: {e}")
        return None

# --- STEP 2: 界面の構築・トリミング・最適化クラス ---
class InterfaceOptimizer:
    def __init__(self, surfaces_dir, interfaces_dir):
        self.surfaces_dir = Path(surfaces_dir)
        self.interfaces_dir = Path(interfaces_dir)
        self.interfaces_dir.mkdir(parents=True, exist_ok=True)
        print(f"\n--- STEP 2: 界面 構築・トリミング・最適化プロセスを開始 ---")
        print(f"✅ 最終構造の保存先: '{self.interfaces_dir}'")

    def find_and_categorize_surfaces(self):
        """
        [変更] 指定されたAlF3とAl2O3のファイルのみをAl側として取得し、
               正極側ファイルは従来通り検索する。
        """
        print(f"🔍 '{self.surfaces_dir}' から正極材表面 (*_final.xyz) を分類中...")
        if not self.surfaces_dir.is_dir():
            print(f"🛑 エラー: 表面ディレクトリ '{self.surfaces_dir}' が見つかりません。")
            return [], []

        # 正極材ファイルは従来通り検索
        all_surfaces = list(self.surfaces_dir.glob("*_final.xyz"))
        cathode_surfaces = [p for p in all_surfaces if "NMC" in p.stem]

        # ===== ここからが変更点 =====
        # Al側は指定されたファイルのみに限定
        print("... 指定されたAl側ファイル (AlF3, Al2O3) のみ対象とします...")
        al_surfaces = []
        additional_al_files_paths = [
            Path("/home/jovyan/Kaori/MD/LiB_2/AlF3_only.xyz"),
            Path("/home/jovyan/Kaori/MD/LiB_2/Al2O3_only.xyz")
        ]
        
        for file_path in additional_al_files_paths:
            if file_path.exists():
                al_surfaces.append(file_path)
                print(f"  -> ✅ 対象に追加: {file_path.name}")
            else:
                print(f"  -> ⚠️  警告: 指定されたファイルが見つかりません: {file_path}")
        # ===== 変更点ここまで =====

        print(f"  - Al表面 (指定ファイルのみ): {len(al_surfaces)}個,  正極材表面: {len(cathode_surfaces)}個")
        return al_surfaces, cathode_surfaces

    def build_interface(self, slab1, slab2, target_xy=(12.0, 12.0), separation=2.0):
        print(f"    1. 界面を構築中...")
        position1 = slab1.get_positions()
        position2 = slab2.get_positions()
        indices_to_delete1 = np.where((position1[:, 0] >= target_xy[0]) | (position1[:, 1] >= target_xy[1]))[0]
        indices_to_delete2 = np.where((position2[:, 0] >= target_xy[0]) | (position2[:, 1] >= target_xy[1]))[0]
        cut_slab1, cut_slab2 = slab1.copy(), slab2.copy()
        if len(indices_to_delete1) > 0: del cut_slab1[indices_to_delete1]
        if len(indices_to_delete2) > 0: del cut_slab2[indices_to_delete2]
        z1_max = cut_slab1.positions[:, 2].max()
        z2_min = cut_slab2.positions[:, 2].min()
        cut_slab2.positions[:, 2] += z1_max - z2_min + separation
        interface = cut_slab1 + cut_slab2
        interface.cell = [12, 12, 53]
        return interface

    def trim_interface(self, atoms, thickness_per_slab=25.0):
        print(f"    2. 界面をトリミング中 (各スラブ厚み: {thickness_per_slab} Å)...")
        al_indices = [a.index for a in atoms if a.symbol in ['Al', 'F']] # AlとFをAl側とみなす
        nmc_indices = [a.index for a in atoms if a.symbol not in ['Al', 'F']]
        if not al_indices or not nmc_indices:
            print("     -> 警告: Al側またはNMC側の原子が見つからず、トリミングをスキップします。")
            return atoms

        al_pos = atoms.get_positions()[al_indices]
        nmc_pos = atoms.get_positions()[nmc_indices]
        
        if np.mean(al_pos[:, 2]) < np.mean(nmc_pos[:, 2]):
            interface_z = (al_pos[:, 2].max() + nmc_pos[:, 2].min()) / 2
            is_al_bottom = True
        else:
            interface_z = (nmc_pos[:, 2].max() + al_pos[:, 2].min()) / 2
            is_al_bottom = False

        indices_to_delete = []
        for atom in atoms:
            is_al_side = atom.symbol in ['Al', 'F']
            if is_al_bottom:
                if is_al_side and atom.z < interface_z - thickness_per_slab:
                    indices_to_delete.append(atom.index)
                elif not is_al_side and atom.z > interface_z + thickness_per_slab:
                    indices_to_delete.append(atom.index)
            else:
                if not is_al_side and atom.z < interface_z - thickness_per_slab:
                    indices_to_delete.append(atom.index)
                elif is_al_side and atom.z > interface_z + thickness_per_slab:
                    indices_to_delete.append(atom.index)

        print(f"        -> {len(indices_to_delete)} 個の原子を削除します。")
        if len(indices_to_delete) > 0:
            del atoms[indices_to_delete]
        return atoms

    def create_trim_and_optimize(self, slab1_path, slab2_path):
        slab1_name = slab1_path.stem.replace('_pos_opt_final', '').replace('_cell_opt_final', '')
        slab2_name = slab2_path.stem.replace('_pos_opt_final', '').replace('_cell_opt_final', '')
        interface_name = f"Interface_{slab1_name}_on_{slab2_name}_trimmed"
        output_path = self.interfaces_dir / f"{interface_name}.cif"

        if output_path.exists():
            print(f"✅ 既存ファイルあり、スキップ: {output_path.name}")
            return

        print(f"\n--- 処理開始: {slab1_name} + {slab2_name} ---")
        try:
            slab1, slab2 = read(str(slab1_path)), read(str(slab2_path))
            interface_built = self.build_interface(slab1, slab2)
            interface_trimmed = self.trim_interface(interface_built, thickness_per_slab=25.0)
            if interface_trimmed is None:
                print("    -> ❌ トリミングに失敗したため、スキップします。")
                return

            print(f"    3. トリミング後の構造を最適化中...")
            traj_path = self.interfaces_dir / f"{interface_name}.traj"
            optimized_structure = run_matlantis_optimization(interface_trimmed, traj_path, fmax=0.05, name=interface_name)

            if optimized_structure:
                optimized_structure.center(vacuum=15.0, axis=2)
                optimized_structure.pbc = (True, True, False)
                print(f"    4. 最終構造をファイルに保存中 -> {output_path.name}")
                write(str(output_path), optimized_structure)
            else:
                print("    -> ❌ 最適化に失敗したため、ファイルは保存されません。")
        except Exception as e:
            print(f"  -> ❌ この界面の作成中に重大なエラーが発生: {e}")

    def run(self):
        al_slabs, cathode_slabs = self.find_and_categorize_surfaces()
        if not al_slabs or not cathode_slabs:
            print(" -> Al表面または正極材表面のファイルが不足しているため、スキップします。")
            return

        print("\n--- Al/NMC界面の組み合わせ処理を開始 ---")
        for al_path in al_slabs:
            for cathode_path in cathode_slabs:
                self.create_trim_and_optimize(al_path, cathode_path)
        print("\n✨ STEP 2 全ての処理が完了。")

def show_results_viewer(results_dir):
    print(f"\n--- 🔬 結果ビューワー: '{results_dir}' ---")
    files = sorted(list(Path(results_dir).glob("*.cif")))
    if not files:
        print("表示できる構造がありません。")
        return
    
    options = {"-- 選択してください --": None}
    options.update({p.name: str(p) for p in files})

    def view_structure(file_path):
        if file_path:
            try:
                atoms = read(file_path)
                view = nv.show_ase(atoms)
                display(view)
            except Exception as e:
                print(f"ファイルの読み込みまたは表示中にエラーが発生しました: {e}")

    widgets.interact(
        view_structure,
        file_path=widgets.Dropdown(options=options, description='Interface:', layout={'width': 'max-content'})
    )

# --- メイン実行部 ---
if __name__ == "__main__":
    # ===== 設定項目 =====
    OPTIMIZATION_OUTPUT_DIR = "/home/jovyan/Kaori/MD/LiB_2/structure/output"
    INTERFACES_DIR = "/home/jovyan/Kaori/MD/LiB_2/structure/output/final_interfaces"
    # ====================

    clear_output(wait=True)

    # ===== 変更点 =====
    # STEP 1: 今回は不要なため実行しない
    print("--- STEP 1: Trajectoryファイルの整理はスキップします ---")
    # cleanup = TrajectoryCleanup(OPTIMIZATION_OUTPUT_DIR)
    # cleanup.run()
    # =================

    # STEP 2: 界面の構築、トリミング、最適化を一括実行
    interface_opt = InterfaceOptimizer(surfaces_dir=OPTIMIZATION_OUTPUT_DIR, interfaces_dir=INTERFACES_DIR)
    interface_opt.run()

    print("\n\n✨ 全てのワークフローが完了しました。 ✨")

    # 結果の表示
    show_results_viewer(INTERFACES_DIR)

--- STEP 1: Trajectoryファイルの整理はスキップします ---

--- STEP 2: 界面 構築・トリミング・最適化プロセスを開始 ---
✅ 最終構造の保存先: '/home/jovyan/Kaori/MD/LiB_2/structure/output/final_interfaces'
🔍 '/home/jovyan/Kaori/MD/LiB_2/structure/output' から正極材表面 (*_final.xyz) を分類中...
... 指定されたAl側ファイル (AlF3, Al2O3) のみ対象とします...
  -> ✅ 対象に追加: AlF3_only.xyz
  -> ✅ 対象に追加: Al2O3_only.xyz
  - Al表面 (指定ファイルのみ): 2個,  正極材表面: 86個

--- Al/NMC界面の組み合わせ処理を開始 ---

--- 処理開始: AlF3_only + NMC111_selective_Co_removal_110_(1, 1, 0) ---
    1. 界面を構築中...
    2. 界面をトリミング中 (各スラブ厚み: 25.0 Å)...
        -> 9 個の原子を削除します。
    3. トリミング後の構造を最適化中...
  -> Matlantis最適化開始 (Interface_AlF3_only_on_NMC111_selective_Co_removal_110_(1, 1, 0)_trimmed, fmax = 0.05) ...


  0%|          | 0/5001 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [1]:
#!/usr/bin/env python3
"""
Interface construction and optimization for Al/NMC systems from CIF files.

Key Features:
- Stacks the Al slab in the z-direction to achieve a desired thickness (~30 Å).
- Trims the final interface so both Al and NMC layers are ~30 Å thick.
- Removes the vacuum layer after optimization to create a fully periodic model.
- Uses CIF files as input for both Al and NMC structures.
"""
import os
import numpy as np
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display, clear_output
import nglview as nv

# ASE (Atomic Simulation Environment)
from ase.io import read, write
from ase.build import stack

# Matlantis / PFP
from matlantis_features.atoms import MatlantisAtoms
from matlantis_features.features.common.opt import FireLBFGSASEOptFeature
from matlantis_features.utils.calculators import pfp_estimator_fn
from pfp_api_client.pfp.estimator import EstimatorCalcMode

# --- Matlantis Optimization Function (Unchanged) ---
def run_matlantis_optimization(atoms, trajectory_path, fmax=0.05, name="structure"):
    """Runs a geometry optimization using Matlantis."""
    print(f"  -> Starting Matlantis optimization for {name} (fmax = {fmax})...")
    matlantis_atoms = MatlantisAtoms(atoms)
    estimator_function = pfp_estimator_fn(
        model_version='v7.0.0', calc_mode=EstimatorCalcMode.CRYSTAL_PLUS_D3
    )
    position_optimizer = FireLBFGSASEOptFeature(
        estimator_fn=estimator_function, filter=False, trajectory=str(trajectory_path),
        n_run=5000, fmax=fmax, show_progress_bar=True
    )
    try:
        result = position_optimizer(matlantis_atoms)
        optimized_atoms = result.atoms.ase_atoms
        final_energy = result.output.energy_log[-1]
        print(f"  -> ✔️ Optimization complete! Final Energy: {final_energy:.3f} eV")
        xyz_save_path = Path(trajectory_path).with_suffix('.xyz')
        print(f"  -> 💾 Saving intermediate optimized structure to: {xyz_save_path.name}")
        write(str(xyz_save_path), optimized_atoms)
        return optimized_atoms
    except Exception as e:
        print(f"  -> ❌ An error occurred during optimization: {e}")
        return None

# --- Interface Construction Class ---
class InterfaceOptimizer:
    def __init__(self, al_cif_path, nmc_cif_dir, interfaces_dir):
        self.al_cif_path = Path(al_cif_path)
        self.nmc_cif_dir = Path(nmc_cif_dir)
        self.interfaces_dir = Path(interfaces_dir)
        self.interfaces_dir.mkdir(parents=True, exist_ok=True)
        print("\n--- Interface Construction, Trimming, and Optimization ---")
        print(f"Aluminum Slab: '{self.al_cif_path.name}'")
        print(f"NMC Slabs from: '{self.nmc_cif_dir}'")
        print(f"Output Directory: '{self.interfaces_dir}'")

    def find_surfaces(self):
        """Finds the input Al and NMC CIF files."""
        print("\n🔍 Searching for input structure files...")
        # Find Al slab
        if not self.al_cif_path.is_file():
            print(f"🛑 Error: Aluminum CIF file not found at '{self.al_cif_path}'")
            al_slabs = []
        else:
            al_slabs = [self.al_cif_path]
            print(f"  - Found 1 Al slab: {self.al_cif_path.name}")

        # Find NMC slabs
        if not self.nmc_cif_dir.is_dir():
            print(f"🛑 Error: NMC directory not found at '{self.nmc_cif_dir}'")
            cathode_slabs = []
        else:
            cathode_slabs = sorted(list(self.nmc_cif_dir.glob("NMC*.cif")))
            print(f"  - Found {len(cathode_slabs)} NMC slabs.")
        
        return al_slabs, cathode_slabs

    def _stack_slab_to_thickness(self, atoms, target_thickness=30.0):
        """Stacks a unit cell in the z-direction to reach a target thickness."""
        z_coords = atoms.get_positions()[:, 2]
        cell_thickness = z_coords.max() - z_coords.min()
        if cell_thickness <= 0: return atoms # Avoid division by zero

        num_repeats = int(np.ceil(target_thickness / cell_thickness))
        if num_repeats <= 1:
            return atoms

        print(f"      -> Stacking slab {num_repeats} times to reach ~{target_thickness} Å...")
        stacked_slab = atoms.copy()
        for _ in range(num_repeats - 1):
            stacked_slab = stack(stacked_slab, atoms, axis=2)
        
        final_thickness = stacked_slab.get_positions()[:, 2].max() - stacked_slab.get_positions()[:, 2].min()
        print(f"      -> Created slab with final thickness: {final_thickness:.2f} Å.")
        return stacked_slab

    def build_interface(self, al_slab, nmc_slab, separation=2.0):
        """Builds the interface by stacking and aligning the two slabs."""
        print("    1. Building interface structure...")
        
        # Stack the Al slab to be thicker
        thick_al_slab = self._stack_slab_to_thickness(al_slab, target_thickness=30.0)

        # Align and stack the slabs
        z_al_max = thick_al_slab.positions[:, 2].max()
        z_nmc_min = nmc_slab.positions[:, 2].min()
        nmc_slab.positions[:, 2] += z_al_max - z_nmc_min + separation
        
        interface = thick_al_slab + nmc_slab
        # Set a temporary large cell for optimization
        interface.cell[2, 2] = 100.0
        interface.center(vacuum=15.0, axis=2)
        return interface

    def trim_interface(self, atoms, thickness_per_slab=30.0):
        """Trims the interface to the desired thickness for each material."""
        print(f"    2. Trimming interface (target per slab: {thickness_per_slab} Å)...")
        al_indices = [a.index for a in atoms if a.symbol == 'Al']
        nmc_indices = [a.index for a in atoms if a.symbol != 'Al']

        if not al_indices or not nmc_indices:
            print("       -> ⚠️ Warning: Could not find both Al and NMC atoms. Skipping trim.")
            return atoms

        al_pos = atoms.get_positions()[al_indices]
        nmc_pos = atoms.get_positions()[nmc_indices]
        
        # Determine interface position and which slab is on the bottom
        if np.mean(al_pos[:, 2]) < np.mean(nmc_pos[:, 2]):
            interface_z = (al_pos[:, 2].max() + nmc_pos[:, 2].min()) / 2.0
            is_al_bottom = True
        else:
            interface_z = (nmc_pos[:, 2].max() + al_pos[:, 2].min()) / 2.0
            is_al_bottom = False

        indices_to_delete = []
        for atom in atoms:
            is_al_atom = atom.symbol == 'Al'
            if is_al_bottom:
                if is_al_atom and atom.z < (interface_z - thickness_per_slab):
                    indices_to_delete.append(atom.index)
                elif not is_al_atom and atom.z > (interface_z + thickness_per_slab):
                    indices_to_delete.append(atom.index)
            else: # NMC is on the bottom
                if not is_al_atom and atom.z < (interface_z - thickness_per_slab):
                    indices_to_delete.append(atom.index)
                elif is_al_atom and atom.z > (interface_z + thickness_per_slab):
                    indices_to_delete.append(atom.index)
        
        print(f"        -> Deleting {len(indices_to_delete)} atoms outside the target thickness.")
        if indices_to_delete:
            del atoms[indices_to_delete]
        return atoms

    def process_interface(self, al_path, nmc_path):
        """The main workflow for one pair of Al and NMC slabs."""
        al_name = al_path.stem
        nmc_name = nmc_path.stem
        interface_name = f"Interface_{al_name}_on_{nmc_name}"
        output_path = self.interfaces_dir / f"{interface_name}.cif"

        if output_path.exists():
            print(f"✅ Output file already exists, skipping: {output_path.name}")
            return

        print(f"\n--- Processing combination: {al_name} + {nmc_name} ---")
        try:
            al_slab = read(al_path)
            nmc_slab = read(nmc_path)
            
            # Step 1: Build and Trim
            interface_built = self.build_interface(al_slab, nmc_slab)
            interface_trimmed = self.trim_interface(interface_built, thickness_per_slab=30.0)

            # Step 2: Optimize
            print("    3. Optimizing atomic positions...")
            traj_path = self.interfaces_dir / f"{interface_name}_opt.traj"
            optimized_atoms = run_matlantis_optimization(interface_trimmed, traj_path, name=interface_name)

            # Step 3: Finalize and Save
            if optimized_atoms:
                print("    4. Finalizing structure (removing vacuum)...")
                # Remove vacuum layer by fitting the cell to the atoms
                positions = optimized_atoms.get_positions()
                z_min, z_max = positions[:, 2].min(), positions[:, 2].max()
                optimized_atoms.cell[2, 2] = z_max - z_min
                positions[:, 2] -= z_min # Shift atoms to start at z=0
                optimized_atoms.set_positions(positions)
                
                # Set periodic boundary conditions in all directions
                optimized_atoms.pbc = (True, True, True)

                print(f"       -> Saving final periodic structure to: {output_path.name}")
                write(str(output_path), optimized_atoms)
            else:
                print("    -> ❌ Optimization failed, structure not saved.")

        except Exception as e:
            print(f"  -> ❌ A critical error occurred while processing this interface: {e}")

    def run(self):
        """Runs the entire interface generation workflow."""
        al_slabs, cathode_slabs = self.find_surfaces()
        if not al_slabs or not cathode_slabs:
            print("\n-> Could not find Al or NMC slabs. Aborting workflow.")
            return

        for al_path in al_slabs:
            for nmc_path in cathode_slabs:
                self.process_interface(al_path, nmc_path)
        
        print("\n✨ All interface processing is complete. ✨")

# --- Results Viewer Function (Unchanged) ---
def show_results_viewer(results_dir):
    """Creates an interactive dropdown to view the generated CIF files."""
    print(f"\n--- 🔬 Results Viewer: '{results_dir}' ---")
    files = sorted(list(Path(results_dir).glob("*.cif")))
    if not files:
        print("No structures available to display.")
        return
    
    options = {"-- Select a structure --": ""}
    options.update({p.name: str(p) for p in files})

    def view_structure(file_path):
        if file_path:
            try:
                atoms = read(file_path)
                view = nv.show_ase(atoms)
                display(view)
            except Exception as e:
                print(f"Error reading or displaying file: {e}")

    widgets.interact(
        view_structure,
        file_path=widgets.Dropdown(options=options, description='Interface:', layout={'width': 'max-content'})
    )

# --- Main Execution Block ---
if __name__ == "__main__":
    # ===== USER SETTINGS =====
    # 1. Path to your single Aluminum (Al) CIF file
    AL_CIF_PATH = "/home/jovyan/Kaori/MD/LiB_2/Al555.xyz"

    # 2. Path to the directory containing your Nickel Manganese Cobalt Oxide (NMC) CIF files
    NMC_CIF_DIR = "/home/jovyan/Kaori/MD/LiB_2/structure/output/cif_files/cif_files/"

    # 3. Directory where the final interface structures will be saved
    INTERFACES_DIR = "/home/jovyan/Kaori/MD/LiB_2/structure/output/final_interfaces_Al_NMC"
    # =========================

    clear_output(wait=True)

    # Initialize and run the interface optimization workflow
    interface_optimizer = InterfaceOptimizer(
        al_cif_path=AL_CIF_PATH,
        nmc_cif_dir=NMC_CIF_DIR,
        interfaces_dir=INTERFACES_DIR
    )
    interface_optimizer.run()

    # Display the results
    show_results_viewer(INTERFACES_DIR)


--- Interface Construction, Trimming, and Optimization ---
Aluminum Slab: 'Al555.xyz'
NMC Slabs from: '/home/jovyan/Kaori/MD/LiB_2/structure/output/cif_files/cif_files'
Output Directory: '/home/jovyan/Kaori/MD/LiB_2/structure/output/final_interfaces_Al_NMC'

🔍 Searching for input structure files...
  - Found 1 Al slab: Al555.xyz
  - Found 17 NMC slabs.

--- Processing combination: Al555 + NMC_Co_decrease_001_surface ---
    1. Building interface structure...
      -> Stacking slab 4 times to reach ~30.0 Å...
      -> Created slab with final thickness: 34.40 Å.
    2. Trimming interface (target per slab: 30.0 Å)...
        -> Deleting 75 atoms outside the target thickness.
    3. Optimizing atomic positions...
  -> Starting Matlantis optimization for Interface_Al555_on_NMC_Co_decrease_001_surface (fmax = 0.05)...


  0%|          | 0/5001 [00:00<?, ?it/s]

  -> ❌ An error occurred during optimization: Invalid input value is detected: coordinates (positions) contains infinite values.
=== When reporting an error, please also share the data below: ===
----------------------------------------------------------------------------
time: 2025-10-14T08:27:37.020177+00:00
pid: 27
code: INVALID_ARGUMENT
method: /pfp.Estimator/Estimate
details: Invalid=20input=20value=20is=20detected:=20coordinates=20(position=
s)=20contains=20infinite=20values.
exc: <_InactiveRpcError=20of=20RPC=20that=20terminated=20with:
=09status=20=3D=20StatusCode.INVALID_ARGUMENT
=09details=20=3D=20"Invalid=20input=20value=20is=20detected:=20coordinates=
=20(positions)=20contains=20infinite=20values."
=09debug_error_string=20=3D=20"UNKNOWN:Error=20received=20from=20peer=20=20=
{grpc_status:3,=20grpc_message:"Invalid=20input=20value=20is=20detected:=20=
coordinates=20(positions)=20contains=20infinite=20values."}"
>
notebook_id: ez65y7njhez5t7bp
metadata: [('client-process-id',=

  0%|          | 0/5001 [00:00<?, ?it/s]

  -> ❌ An error occurred during optimization: Invalid input value is detected: coordinates (positions) contains infinite values.
=== When reporting an error, please also share the data below: ===
----------------------------------------------------------------------------
time: 2025-10-14T08:27:56.642476+00:00
pid: 27
code: INVALID_ARGUMENT
method: /pfp.Estimator/Estimate
details: Invalid=20input=20value=20is=20detected:=20coordinates=20(position=
s)=20contains=20infinite=20values.
exc: <_InactiveRpcError=20of=20RPC=20that=20terminated=20with:
=09status=20=3D=20StatusCode.INVALID_ARGUMENT
=09details=20=3D=20"Invalid=20input=20value=20is=20detected:=20coordinates=
=20(positions)=20contains=20infinite=20values."
=09debug_error_string=20=3D=20"UNKNOWN:Error=20received=20from=20peer=20=20=
{grpc_message:"Invalid=20input=20value=20is=20detected:=20coordinates=20(po=
sitions)=20contains=20infinite=20values.",=20grpc_status:3}"
>
notebook_id: ez65y7njhez5t7bp
metadata: [('client-process-id',=

KeyboardInterrupt: 

In [2]:
#!/usr/bin/env python3
"""
界面構築・最適化を自動化するプログラム。

[最終版: CIF対応]
- Material 1 を特定のファイルから読み込み、Material 2 をCIFファイルディレクトリから検索して界面を構築します。
- 破損したファイルなど、特定ファイルの処理でエラーが発生しても全体が停止せず、
  次のファイルの処理を続行します。
"""
import os
import itertools
import numpy as np
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display, clear_output
import nglview as nv

# ASE
from ase.io import read, write
from ase.build import stack,cut

# Matlantis / PFP
from matlantis_features.atoms import MatlantisAtoms
from matlantis_features.features.common.opt import FireLBFGSASEOptFeature
from matlantis_features.utils.calculators import pfp_estimator_fn
from pfp_api_client.pfp.estimator import EstimatorCalcMode

# --- Matlantis最適化関数 (変更なし) ---
def run_matlantis_optimization(atoms, trajectory_path, fmax=0.05, name="structure"):
    print(f"  -> Matlantis最適化開始 ({name}, fmax = {fmax}) ...")
    matlantis_atoms = MatlantisAtoms(atoms)
    estimator_function = pfp_estimator_fn(
        model_version='v7.0.0', calc_mode=EstimatorCalcMode.CRYSTAL_PLUS_D3
    )
    position_optimizer = FireLBFGSASEOptFeature(
        estimator_fn=estimator_function, filter=False, trajectory=str(trajectory_path),
        n_run=5000, fmax=fmax, show_progress_bar=True
    )
    try:
        result = position_optimizer(matlantis_atoms)
        optimized_atoms = result.atoms.ase_atoms
        final_energy = result.output.energy_log[-1]
        print(f"  -> ✔️ 最適化完了！ エネルギー: {final_energy:.3f} eV")
        xyz_save_path = Path(trajectory_path).with_suffix('.xyz')
        print(f"  -> 💾 最適化後の中間構造を保存します: {xyz_save_path.name}")
        write(str(xyz_save_path), optimized_atoms)
        return optimized_atoms
    except Exception as e:
        print(f"  -> ❌ 最適化中にエラーが発生: {e}")
        return None

# --- STEP 2: 界面の構築・トリミング・最適化クラス ---
class InterfaceOptimizer:
    def __init__(self, material1_filepath, material2_surfaces_dir, interfaces_dir, material2_keyword):
        self.material1_filepath = Path(material1_filepath)
        self.material2_surfaces_dir = Path(material2_surfaces_dir)
        self.interfaces_dir = Path(interfaces_dir)
        self.material2_keyword = material2_keyword
        
        self.interfaces_dir.mkdir(parents=True, exist_ok=True)
        print(f"\n--- 界面 構築・トリミング・最適化プロセスを開始 ---")
        print(f"   - Material 1 (指定): '{self.material1_filepath.name}'")
        print(f"   - Material 2 (検索): '{self.material2_keyword}' で始まるCIFファイル")
        print(f"   - 保存先: '{self.interfaces_dir}'")

    # ★変更点1: 検索するファイルの拡張子を .cif に変更
    def find_and_categorize_surfaces(self):
        print(f"🔍 表面ファイルを分類中...")
        
        # --- Material 1: 指定されたファイルパスの存在確認 ---
        if not self.material1_filepath.exists():
            print(f"🛑 エラー: 指定されたMaterial1ファイル '{self.material1_filepath}' が見つかりません。")
            surfaces1 = []
        else:
            print(f"  - Material 1: 1個見つかりました -> {self.material1_filepath.name}")
            surfaces1 = [self.material1_filepath]

        # --- Material 2: ディレクトリからキーワードで .cif ファイルを検索 ---
        if not self.material2_surfaces_dir.is_dir():
            print(f"🛑 エラー: Material2の表面ディレクトリ '{self.material2_surfaces_dir}' が見つかりません。")
            surfaces2 = []
        else:
            # ここで *.cif を検索するように変更
            all_surfaces = list(self.material2_surfaces_dir.glob("*.cif"))
            surfaces2 = [p for p in all_surfaces if p.stem.startswith(self.material2_keyword)]
            print(f"  - {self.material2_keyword}表面: {len(surfaces2)}個見つかりました。")
            
        return surfaces1, surfaces2
    
    def build_interface(self, slab1, slab2, target_xy=(12.0, 12.0), separation=2.0):
        print(f"    1. 界面を構築中...")
        position1 = slab1.get_positions()
        position2 = slab2.get_positions()
        indices_to_delete1 = np.where((position1[:, 0] >= target_xy[0]) | (position1[:, 1] >= target_xy[1]))[0]
        indices_to_delete2 = np.where((position2[:, 0] >= target_xy[0]) | (position2[:, 1] >= target_xy[1]))[0]
        cut_slab1, cut_slab2 = slab1.copy(), slab2.copy()
        if len(indices_to_delete1) > 0: del cut_slab1[indices_to_delete1]
        if len(indices_to_delete2) > 0: del cut_slab2[indices_to_delete2]
        z1_max = cut_slab1.positions[:, 2].max()
        z2_min = cut_slab2.positions[:, 2].min()
        cut_slab2.positions[:, 2] += z1_max - z2_min + separation
        interface = cut_slab1 + cut_slab2
        interface.cell = [12, 12, 53] # セルサイズは適宜調整してください
        return interface

    def trim_interface(self, atoms, thickness_per_slab=25.0):
        print(f"    2. 界面をトリミング中 (各スラブ厚み: {thickness_per_slab} Å)...")
        al_indices = [a.index for a in atoms if a.symbol == 'Al']
        other_indices = [a.index for a in atoms if a.symbol not in ['Al']]
        if not al_indices or not other_indices:
             print("     -> 警告: 界面の両側に原子が見つからないため、トリミングをスキップします。")
             return atoms
        al_pos = atoms.get_positions()[al_indices]
        other_pos = atoms.get_positions()[other_indices]
        if np.mean(al_pos[:, 2]) < np.mean(other_pos[:, 2]):
            interface_z = (al_pos[:, 2].max() + other_pos[:, 2].min()) / 2
        else:
            interface_z = (other_pos[:, 2].max() + al_pos[:, 2].min()) / 2
        indices_to_delete = []
        is_al_bottom = np.mean(al_pos[:, 2]) < np.mean(other_pos[:, 2])
        for atom in atoms:
            if is_al_bottom:
                if atom.symbol == 'Al' and atom.z < interface_z - thickness_per_slab:
                    indices_to_delete.append(atom.index)
                elif atom.symbol != 'Al' and atom.z > interface_z + thickness_per_slab:
                    indices_to_delete.append(atom.index)
            else:
                if atom.symbol != 'Al' and atom.z < interface_z - thickness_per_slab:
                    indices_to_delete.append(atom.index)
                elif atom.symbol == 'Al' and atom.z > interface_z + thickness_per_slab:
                    indices_to_delete.append(atom.index)
        print(f"        -> {len(indices_to_delete)} 個の原子を削除します。")
        if len(indices_to_delete) > 0:
            del atoms[indices_to_delete]
        return atoms

    # ★変更点2: ファイル名から不要な部分を削除する処理をよりシンプルに
    def create_trim_and_optimize(self, slab1_path, slab2_path):
        slab1_name = slab1_path.stem 
        slab2_name = slab2_path.stem
        interface_name = f"Interface_{slab1_name}_on_{slab2_name}_trimmed"
        output_path = self.interfaces_dir / f"{interface_name}.cif"
        if output_path.exists():
            print(f"✅ 既存ファイルあり、スキップ: {output_path.name}")
            return
        print(f"\n--- 処理開始: {slab1_name} + {slab2_name} ---")
        try:
            slab1, slab2 = read(str(slab1_path)), read(str(slab2_path))
            interface_built = self.build_interface(slab1, slab2)
            interface_trimmed = self.trim_interface(interface_built, thickness_per_slab=25.0)
            if interface_trimmed is None:
                print("    -> ❌ トリミングに失敗したため、スキップします。")
                return
            print(f"    3. トリミング後の構造を最適化中...")
            traj_path = self.interfaces_dir / f"{interface_name}.traj"
            optimized_structure = run_matlantis_optimization(interface_trimmed, traj_path, fmax=0.05, name=interface_name)
            if optimized_structure:
                optimized_structure.center(vacuum=15.0, axis=2)
                optimized_structure.pbc = (True, True, False)
                print(f"    4. 最終構造をファイルに保存中 -> {output_path.name}")
                write(str(output_path), optimized_structure)
            else:
                print("    -> ❌ 最適化に失敗したため、ファイルは保存されません。")
        except Exception as e:
            print(f"  -> ❌ この界面の作成中に重大なエラーが発生: {e}")

    def run(self):
        slabs1, slabs2 = self.find_and_categorize_surfaces()
        if not slabs1 or not slabs2:
            print(f" -> Material 1のファイル、またはMaterial 2の表面ファイルが不足しているため、処理を終了します。")
            return
        print(f"\n--- {self.material1_filepath.stem}/{self.material2_keyword}界面の組み合わせ処理を開始 ---")
        for slab1_path in slabs1:
            for slab2_path in slabs2:
                self.create_trim_and_optimize(slab1_path, slab2_path)
        print("\n✨ 全ての処理が完了。")

# --- 結果表示関数 (変更なし) ---
def show_results_viewer(results_dir):
    print(f"\n--- 🔬 結果ビューワー: '{results_dir}' ---")
    files = sorted(list(Path(results_dir).glob("*.cif")))
    if not files:
        print("表示できる構造がありません。")
        return
    options = {"-- 選択してください --": None}
    options.update({p.name: p for p in files})
    widgets.interact(
        lambda file_path: display(nv.show_ase(read(file_path)) if file_path else None),
        file_path=widgets.Dropdown(options=options, description='Interface:', layout={'width': 'max-content'})
    )

# --- メイン実行部 ---
if __name__ == "__main__":
    # ===== 設定項目 =====
    # 1. Material 1 のファイルパスを直接指定
    MATERIAL1_FILE_PATH = "/home/jovyan/Kaori/MD/LiB_2/Al555.xyz"
    
    # ★変更点3: Material 2 のCIFファイルが入っているディレクトリのパスを更新
    MATERIAL2_SURFACES_DIR = "/home/jovyan/Kaori/MD/LiB_2/structure/output/cif_files/cif_file"
    
    # 3. Material 2 を見つけるためのキーワード (ファイル名の先頭部分)
    MATERIAL2_KEYWORD = "NMC_"
    
    # 4. 作成した界面構造の保存先ディレクトリ
    INTERFACES_DIR = "/home/jovyan/Kaori/MD/LiB_2/structure/output/final_interfaces_Al555_vs_NMC"
    # ====================

    clear_output(wait=True)

    # STEP 1のTrajectoryファイル整理は不要なため、呼び出しません。

    # STEP 2: 界面の構築、トリミング、最適化を一括実行
    interface_opt = InterfaceOptimizer(
        material1_filepath=MATERIAL1_FILE_PATH,
        material2_surfaces_dir=MATERIAL2_SURFACES_DIR,
        interfaces_dir=INTERFACES_DIR,
        material2_keyword=MATERIAL2_KEYWORD
    )
    interface_opt.run()

    print("\n\n✨ 全てのワークフローが完了しました。 ✨")

    # 結果の表示
    show_results_viewer(INTERFACES_DIR)


--- 界面 構築・トリミング・最適化プロセスを開始 ---
   - Material 1 (指定): 'Al555.xyz'
   - Material 2 (検索): 'NMC_' で始まるCIFファイル
   - 保存先: '/home/jovyan/Kaori/MD/LiB_2/structure/output/final_interfaces_Al555_vs_NMC'
🔍 表面ファイルを分類中...
  - Material 1: 1個見つかりました -> Al555.xyz
  - NMC_表面: 17個見つかりました。

--- Al555/NMC_界面の組み合わせ処理を開始 ---

--- 処理開始: Al555 + NMC_Mn_decrease_104_surface ---
    1. 界面を構築中...
    2. 界面をトリミング中 (各スラブ厚み: 25.0 Å)...
        -> 0 個の原子を削除します。
    3. トリミング後の構造を最適化中...
  -> Matlantis最適化開始 (Interface_Al555_on_NMC_Mn_decrease_104_surface_trimmed, fmax = 0.05) ...


  0%|          | 0/5001 [00:00<?, ?it/s]

  -> ✔️ 最適化完了！ エネルギー: -620.966 eV
  -> 💾 最適化後の中間構造を保存します: Interface_Al555_on_NMC_Mn_decrease_104_surface_trimmed.xyz
    4. 最終構造をファイルに保存中 -> Interface_Al555_on_NMC_Mn_decrease_104_surface_trimmed.cif

--- 処理開始: Al555 + NMC_Mn_increase_001_surface ---
    1. 界面を構築中...
    2. 界面をトリミング中 (各スラブ厚み: 25.0 Å)...
        -> 0 個の原子を削除します。
    3. トリミング後の構造を最適化中...
  -> Matlantis最適化開始 (Interface_Al555_on_NMC_Mn_increase_001_surface_trimmed, fmax = 0.05) ...


  0%|          | 0/5001 [00:00<?, ?it/s]

  -> ✔️ 最適化完了！ エネルギー: -1762.656 eV
  -> 💾 最適化後の中間構造を保存します: Interface_Al555_on_NMC_Mn_increase_001_surface_trimmed.xyz
    4. 最終構造をファイルに保存中 -> Interface_Al555_on_NMC_Mn_increase_001_surface_trimmed.cif

--- 処理開始: Al555 + NMC_Co_decrease_104_surface ---
    1. 界面を構築中...
    2. 界面をトリミング中 (各スラブ厚み: 25.0 Å)...
        -> 0 個の原子を削除します。
    3. トリミング後の構造を最適化中...
  -> Matlantis最適化開始 (Interface_Al555_on_NMC_Co_decrease_104_surface_trimmed, fmax = 0.05) ...


...

  0%|          | 0/5001 [00:00<?, ?it/s]

  -> ✔️ 最適化完了！ エネルギー: -930.500 eV
  -> 💾 最適化後の中間構造を保存します: Interface_Al555_on_NMC_Mn_decrease_012_surface_trimmed.xyz
    4. 最終構造をファイルに保存中 -> Interface_Al555_on_NMC_Mn_decrease_012_surface_trimmed.cif

--- 処理開始: Al555 + NMC_Co_decrease_012_surface ---
    1. 界面を構築中...
    2. 界面をトリミング中 (各スラブ厚み: 25.0 Å)...
        -> 0 個の原子を削除します。
    3. トリミング後の構造を最適化中...
  -> Matlantis最適化開始 (Interface_Al555_on_NMC_Co_decrease_012_surface_trimmed, fmax = 0.05) ...


  0%|          | 0/5001 [00:00<?, ?it/s]

  -> ✔️ 最適化完了！ エネルギー: -946.308 eV
  -> 💾 最適化後の中間構造を保存します: Interface_Al555_on_NMC_Co_decrease_012_surface_trimmed.xyz
    4. 最終構造をファイルに保存中 -> Interface_Al555_on_NMC_Co_decrease_012_surface_trimmed.cif

✨ 全ての処理が完了。


✨ 全てのワークフローが完了しました。 ✨

--- 🔬 結果ビューワー: '/home/jovyan/Kaori/MD/LiB_2/structure/output/final_interfaces_Al555_vs_NMC' ---


interactive(children=(Dropdown(description='Interface:', layout=Layout(width='max-content'), options={'-- 選択して…

In [6]:
from pfcc_extras import show_gui, view_ngl
from ase.io import Trajectory,read,write
import os
import itertools
import numpy as np
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display, clear_output
import nglview as nv

# ASE
from ase.io import read, write
from ase.build import stack,cut,bulk,fcc100

atoms = fcc100("Al",size = (5, 5, 10),a=4.0)
# show_gui(atoms)
# write("/home/jovyan/Kaori/MD/LiB_2/Al555.xyz",atoms)
atoms.cell

Cell([14.142135623730951, 14.142135623730951, 0.0])

In [2]:
#!/usr/bin/env python3
"""
Trajectoryファイルの整理と、それに基づく界面構築・最適化を自動化するプログラム。

[追加実行版]
- AlF3とAl2O3のみをAl側表面として扱い、追加で界面構築と最適化を実行します。
- STEP1のTrajectoryファイル整理はスキップします。
"""
import os
import itertools
import numpy as np
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display, clear_output
import nglview as nv

# ASE
from ase.io import read, write
from ase.build import stack,cut

# Matlantis / PFP
from matlantis_features.atoms import MatlantisAtoms
from matlantis_features.features.common.opt import FireLBFGSASEOptFeature
from matlantis_features.utils.calculators import pfp_estimator_fn
from pfp_api_client.pfp.estimator import EstimatorCalcMode

# --- STEP 1: Trajectoryファイルの整理クラス (今回は使用しない) ---
class TrajectoryCleanup:
    def __init__(self, directory):
        print("--- STEP 1: Trajectoryファイルの分析と整理 (スキップ) ---")
    def run(self):
        pass

# --- Matlantis最適化関数 (共通) ---

def run_matlantis_optimization(atoms, trajectory_path, fmax=0.05, name="structure"):
    print(f"  -> Matlantis最適化開始 ({name}, fmax = {fmax}) ...")
    matlantis_atoms = MatlantisAtoms(atoms)
    estimator_function = pfp_estimator_fn(
        model_version='v7.0.0', calc_mode=EstimatorCalcMode.CRYSTAL_PLUS_D3
    )
    position_optimizer = FireLBFGSASEOptFeature(
        estimator_fn=estimator_function, filter=False, trajectory=str(trajectory_path),
        n_run=5000, fmax=fmax, show_progress_bar=True
    )
    try:
        result = position_optimizer(matlantis_atoms)
        optimized_atoms = result.atoms.ase_atoms
        final_energy = result.output.energy_log[-1]
        print(f"  -> ✔️ 最適化完了！ エネルギー: {final_energy:.3f} eV")
        xyz_save_path = Path(trajectory_path).with_suffix('.xyz')
        print(f"  -> 💾 最適化後の中間構造を保存します: {xyz_save_path.name}")
        write(str(xyz_save_path), optimized_atoms)
        return optimized_atoms
    except Exception as e:
        print(f"  -> ❌ 最適化中にエラーが発生: {e}")
        return None

# --- STEP 2: 界面の構築・トリミング・最適化クラス ---
class InterfaceOptimizer:
    def __init__(self, surfaces_dir, interfaces_dir):
        self.surfaces_dir = Path(surfaces_dir)
        self.interfaces_dir = Path(interfaces_dir)
        self.interfaces_dir.mkdir(parents=True, exist_ok=True)
        print(f"\n--- STEP 2: 界面 構築・トリミング・最適化プロセスを開始 ---")
        print(f"✅ 最終構造の保存先: '{self.interfaces_dir}'")

    def find_and_categorize_surfaces(self):
        """
        [変更] 指定されたAlF3とAl2O3のファイルのみをAl側として取得し、
               正極側ファイルは従来通り検索する。
        """
        print(f"🔍 '{self.surfaces_dir}' から正極材表面 (*_final.xyz) を分類中...")
        if not self.surfaces_dir.is_dir():
            print(f"🛑 エラー: 表面ディレクトリ '{self.surfaces_dir}' が見つかりません。")
            return [], []

        # 正極材ファイルは従来通り検索
        all_surfaces = list(self.surfaces_dir.glob("*_final.xyz"))
        cathode_surfaces = [p for p in all_surfaces if "NMC" in p.stem]

        # ===== ここからが変更点 =====
        # Al側は指定されたファイルのみに限定
        print("... 指定されたAl側ファイル (AlF3, Al2O3) のみ対象とします...")
        al_surfaces = []
        additional_al_files_paths = [
            Path("/home/jovyan/Kaori/MD/LiB_2/Al555.xyz")
            # Path("/home/jovyan/Kaori/MD/LiB_2/AlF3_only.xyz"),
            # Path("/home/jovyan/Kaori/MD/LiB_2/Al2O3_only.xyz")
        ]
        
        for file_path in additional_al_files_paths:
            if file_path.exists():
                al_surfaces.append(file_path)
                print(f"  -> ✅ 対象に追加: {file_path.name}")
            else:
                print(f"  -> ⚠️  警告: 指定されたファイルが見つかりません: {file_path}")
        # ===== 変更点ここまで =====

        print(f"  - Al表面 (指定ファイルのみ): {len(al_surfaces)}個,  正極材表面: {len(cathode_surfaces)}個")
        return al_surfaces, cathode_surfaces

    def build_interface(self, slab1, slab2, target_xy=(12.0, 12.0), separation=2.0):
        print(f"    1. 界面を構築中...")
        position1 = slab1.get_positions()
        position2 = slab2.get_positions()
        indices_to_delete1 = np.where((position1[:, 0] >= target_xy[0]) | (position1[:, 1] >= target_xy[1]))[0]
        indices_to_delete2 = np.where((position2[:, 0] >= target_xy[0]) | (position2[:, 1] >= target_xy[1]))[0]
        cut_slab1, cut_slab2 = slab1.copy(), slab2.copy()
        if len(indices_to_delete1) > 0: del cut_slab1[indices_to_delete1]
        if len(indices_to_delete2) > 0: del cut_slab2[indices_to_delete2]
        z1_max = cut_slab1.positions[:, 2].max()
        z2_min = cut_slab2.positions[:, 2].min()
        cut_slab2.positions[:, 2] += z1_max - z2_min + separation
        interface = cut_slab1 + cut_slab2
        interface.cell = [12, 12, 53]
        return interface

    def trim_interface(self, atoms, thickness_per_slab=25.0):
        print(f"    2. 界面をトリミング中 (各スラブ厚み: {thickness_per_slab} Å)...")
        al_indices = [a.index for a in atoms if a.symbol in ['Al', 'F']] # AlとFをAl側とみなす
        nmc_indices = [a.index for a in atoms if a.symbol not in ['Al', 'F']]
        if not al_indices or not nmc_indices:
            print("     -> 警告: Al側またはNMC側の原子が見つからず、トリミングをスキップします。")
            return atoms

        al_pos = atoms.get_positions()[al_indices]
        nmc_pos = atoms.get_positions()[nmc_indices]
        
        if np.mean(al_pos[:, 2]) < np.mean(nmc_pos[:, 2]):
            interface_z = (al_pos[:, 2].max() + nmc_pos[:, 2].min()) / 2
            is_al_bottom = True
        else:
            interface_z = (nmc_pos[:, 2].max() + al_pos[:, 2].min()) / 2
            is_al_bottom = False

        indices_to_delete = []
        for atom in atoms:
            is_al_side = atom.symbol in ['Al', 'F']
            if is_al_bottom:
                if is_al_side and atom.z < interface_z - thickness_per_slab:
                    indices_to_delete.append(atom.index)
                elif not is_al_side and atom.z > interface_z + thickness_per_slab:
                    indices_to_delete.append(atom.index)
            else:
                if not is_al_side and atom.z < interface_z - thickness_per_slab:
                    indices_to_delete.append(atom.index)
                elif is_al_side and atom.z > interface_z + thickness_per_slab:
                    indices_to_delete.append(atom.index)

        print(f"        -> {len(indices_to_delete)} 個の原子を削除します。")
        if len(indices_to_delete) > 0:
            del atoms[indices_to_delete]
        return atoms

    def create_trim_and_optimize(self, slab1_path, slab2_path):
        slab1_name = slab1_path.stem.replace('_pos_opt_final', '').replace('_cell_opt_final', '')
        slab2_name = slab2_path.stem.replace('_pos_opt_final', '').replace('_cell_opt_final', '')
        interface_name = f"Interface_{slab1_name}_on_{slab2_name}_trimmed"
        output_path = self.interfaces_dir / f"{interface_name}.cif"

        if output_path.exists():
            print(f"✅ 既存ファイルあり、スキップ: {output_path.name}")
            return

        print(f"\n--- 処理開始: {slab1_name} + {slab2_name} ---")
        try:
            slab1, slab2 = read(str(slab1_path)), read(str(slab2_path))
            interface_built = self.build_interface(slab1, slab2)
            interface_trimmed = self.trim_interface(interface_built, thickness_per_slab=25.0)
            if interface_trimmed is None:
                print("    -> ❌ トリミングに失敗したため、スキップします。")
                return

            print(f"    3. トリミング後の構造を最適化中...")
            traj_path = self.interfaces_dir / f"{interface_name}.traj"
            optimized_structure = run_matlantis_optimization(interface_trimmed, traj_path, fmax=0.05, name=interface_name)

            if optimized_structure:
                optimized_structure.center(vacuum=15.0, axis=2)
                optimized_structure.pbc = (True, True, False)
                print(f"    4. 最終構造をファイルに保存中 -> {output_path.name}")
                write(str(output_path), optimized_structure)
            else:
                print("    -> ❌ 最適化に失敗したため、ファイルは保存されません。")
        except Exception as e:
            print(f"  -> ❌ この界面の作成中に重大なエラーが発生: {e}")

    def run(self):
        al_slabs, cathode_slabs = self.find_and_categorize_surfaces()
        if not al_slabs or not cathode_slabs:
            print(" -> Al表面または正極材表面のファイルが不足しているため、スキップします。")
            return

        print("\n--- Al/NMC界面の組み合わせ処理を開始 ---")
        for al_path in al_slabs:
            for cathode_path in cathode_slabs:
                self.create_trim_and_optimize(al_path, cathode_path)
        print("\n✨ STEP 2 全ての処理が完了。")

def show_results_viewer(results_dir):
    print(f"\n--- 🔬 結果ビューワー: '{results_dir}' ---")
    files = sorted(list(Path(results_dir).glob("*.cif")))
    if not files:
        print("表示できる構造がありません。")
        return
    
    options = {"-- 選択してください --": None}
    options.update({p.name: str(p) for p in files})

    def view_structure(file_path):
        if file_path:
            try:
                atoms = read(file_path)
                view = nv.show_ase(atoms)
                display(view)
            except Exception as e:
                print(f"ファイルの読み込みまたは表示中にエラーが発生しました: {e}")

    widgets.interact(
        view_structure,
        file_path=widgets.Dropdown(options=options, description='Interface:', layout={'width': 'max-content'})
    )

# --- メイン実行部 ---
if __name__ == "__main__":
    # ===== 設定項目 =====
    OPTIMIZATION_OUTPUT_DIR = "/home/jovyan/Kaori/MD/LiB_2/structure/output"
    INTERFACES_DIR = "/home/jovyan/Kaori/MD/LiB_2/structure/output/final_interfaces2"
    # ====================

    clear_output(wait=True)

    # ===== 変更点 =====
    # STEP 1: 今回は不要なため実行しない
    print("--- STEP 1: Trajectoryファイルの整理はスキップします ---")
    # cleanup = TrajectoryCleanup(OPTIMIZATION_OUTPUT_DIR)
    # cleanup.run()
    # =================

    # STEP 2: 界面の構築、トリミング、最適化を一括実行
    interface_opt = InterfaceOptimizer(surfaces_dir=OPTIMIZATION_OUTPUT_DIR, interfaces_dir=INTERFACES_DIR)
    interface_opt.run()

    print("\n\n✨ 全てのワークフローが完了しました。 ✨")

    # 結果の表示
    show_results_viewer(INTERFACES_DIR)

--- STEP 1: Trajectoryファイルの整理はスキップします ---

--- STEP 2: 界面 構築・トリミング・最適化プロセスを開始 ---
✅ 最終構造の保存先: '/home/jovyan/Kaori/MD/LiB_2/structure/output/final_interfaces2'
🔍 '/home/jovyan/Kaori/MD/LiB_2/structure/output' から正極材表面 (*_final.xyz) を分類中...
... 指定されたAl側ファイル (AlF3, Al2O3) のみ対象とします...
  -> ✅ 対象に追加: Al555.xyz
  - Al表面 (指定ファイルのみ): 1個,  正極材表面: 0個
 -> Al表面または正極材表面のファイルが不足しているため、スキップします。


✨ 全てのワークフローが完了しました。 ✨

--- 🔬 結果ビューワー: '/home/jovyan/Kaori/MD/LiB_2/structure/output/final_interfaces2' ---
表示できる構造がありません。


In [1]:
!pip install --force-reinstall pfp-api-client

Looking in indexes: https://pypi.org/simple, http://pypi.artifact.svc:8080/simple
  Using cached pfp_api_client-1.24.0-py3-none-any.whl
  Using cached ase-3.26.0-py3-none-any.whl.metadata (4.1 kB)
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached grpcio-1.75.1-cp311-cp311-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (3.7 kB)
  Using cached protobuf-5.29.5-cp38-abi3-manylinux2014_x86_64.whl.metadata (592 bytes)
  Using cached googleapis_common_protos-1.70.0-py3-none-any.whl.metadata (9.3 kB)
  Using cached scipy-1.16.2-cp311-cp311-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (62 kB)
  Using cached contourpy-1.3.3-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached kiwisolver-1.4.9-cp311-cp311-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (6.3 kB)
  Using cached packaging-25.0-py3-none

In [68]:
#!/usr/bin/env python3
"""
Trajectoryファイルの整理と、それに基づく界面構築・最適化を自動化するプログラム。

[エラー耐性強化版]
- 破損したファイルなど、特定ファイルの処理でエラーが発生しても全体が停止せず、
  次のファイルの処理を続行します。

STEP 1: Trajectoryファイルの分析とクリーンアップ
  - 0KBの.trajファイル（失敗した計算）を削除
  - 正常な.trajファイルを初期/最終構造の.xyzに変換し、元ファイルを削除

STEP 2: 界面の構築と最適化
  - STEP 1で生成された最終構造(.xyz)を用いて界面を構築
  - Matlantisを用いて原子位置のみを最適化
"""
import os
import itertools
import numpy as np
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display, clear_output
import nglview as nv

# ASE
from ase.io import read, write
from ase.build import stack,cut

# Matlantis / PFP
from matlantis_features.atoms import MatlantisAtoms
from matlantis_features.features.common.opt import FireLBFGSASEOptFeature
from matlantis_features.utils.calculators import pfp_estimator_fn
from pfp_api_client.pfp.estimator import EstimatorCalcMode

# --- STEP 1: Trajectoryファイルの整理クラス ---
class TrajectoryCleanup:
    """
    .trajファイルを分析し、.xyzに変換してクリーンアップするクラス。
    """
    def __init__(self, directory):
        self.directory = Path(directory)
        print("--- STEP 1: Trajectoryファイルの分析と整理を開始 ---")
        if not self.directory.is_dir():
            print(f"🛑 エラー: ディレクトリ '{self.directory}' が見つかりません。")
            self.files_to_process = []
        else:
            self.files_to_process = list(self.directory.glob("*.traj"))
            print(f"🔍 '{self.directory}' から {len(self.files_to_process)} 個の .traj ファイルを発見。")

    def process_file(self, traj_path):
        """
        1つの.trajファイルを処理する。
        [修正] この関数全体をtry...exceptで囲み、エラー耐性を向上。
        """
        try:
            # ファイルサイズを確認
            file_size = traj_path.stat().st_size
            if file_size == 0:
                print(f"  -> 🗑️  0KBファイルを削除します: {traj_path.name}")
                os.remove(traj_path)
                return

            # 正常なファイルは変換
            base_name = traj_path.stem
            initial_xyz_path = self.directory / f"{base_name}_initial.xyz"
            final_xyz_path = self.directory / f"{base_name}_final.xyz"
            
            print(f"  -> 🔄 変換中: {traj_path.name}")
            
            # Trajectoryから初期構造と最終構造を読み込み
            atoms_list = read(traj_path, index=":")
            if not atoms_list:
                print(f"  -> ❌ 読み込みエラー（空のTrajectory）、ファイルを削除します: {traj_path.name}")
                os.remove(traj_path)
                return

            # .xyzファイルとして保存
            write(str(initial_xyz_path), atoms_list[0])
            write(str(final_xyz_path), atoms_list[-1])
            
            # 元の.trajファイルを削除
            os.remove(traj_path)
            print(f"  -> ✔️ 変換成功、元のファイルを削除しました: {traj_path.name}")

        except Exception as e:
            # 予期せぬエラーが発生した場合、ログを出力して処理を続行
            print(f"  -> ❌ 重大なエラーが発生したため、このファイルはスキップします ({traj_path.name}): {e}")

    def run(self):
        """全ての対象ファイルに対して処理を実行する。"""
        if not self.files_to_process:
            print(" -> 処理対象の.trajファイルがないため、スキップします。")
            return
        for traj_file in self.files_to_process:
            self.process_file(traj_file)
        print("\n✨ STEP 1 完了。")


# --- Matlantis最適化関数 (共通) ---
def run_matlantis_optimization(atoms, trajectory_path, fmax=0.05, name="structure"):
    print(f"  -> Matlantis最適化開始 ({name}, fmax = {fmax}) ...")
    matlantis_atoms = MatlantisAtoms(atoms)
    estimator_function = pfp_estimator_fn(
        model_version='v7.0.0', calc_mode=EstimatorCalcMode.CRYSTAL_PLUS_D3
    )
    position_optimizer = FireLBFGSASEOptFeature(
        estimator_fn=estimator_function, filter=False, trajectory=str(trajectory_path),
        n_run=5000, fmax=fmax, show_progress_bar=True
    )
    try:
        result = position_optimizer(matlantis_atoms)
        optimized_atoms = result.atoms.ase_atoms
        final_energy = result.output.energy_log[-1]
        print(f"  -> ✔️ 最適化完了！ エネルギー: {final_energy:.3f} eV")
        xyz_save_path = Path(trajectory_path).with_suffix('.xyz')
        print(f"  -> 💾 最適化後の構造を保存します: {xyz_save_path.name}")
        write(str(xyz_save_path), optimized_atoms)
        return True
    except Exception as e:
        print(f"  -> ❌ 最適化中にエラーが発生: {e}")
        return False

# --- STEP 2: 界面最適化クラス ---
class InterfaceOptimizer:
    def __init__(self, surfaces_dir, interfaces_dir):
        self.surfaces_dir = Path(surfaces_dir)
        self.interfaces_dir = Path(interfaces_dir)
        self.interfaces_dir.mkdir(parents=True, exist_ok=True)
        print(f"\n--- STEP 2: 界面構築・最適化プロセスを開始 ---")
        print(f"✅ 界面構造の保存先: '{self.interfaces_dir}'")

    # def build_and_cut_interface(self, slab1, slab2, target_xy=(12.0, 12.0), target_z=20.0, separation=2.0):
    #     """
    #     緩和済み表面を維持しつつ、指定サイズに界面を精密に構築・カットする。
    #     """
    #     print(f"    -> 界面を ({target_xy[0]}, {target_xy[1]}, {target_z}) Å に精密構築...")
    #     # Step 1: 各スラブをX,Y方向にカット
    #     # cut_slab1 = cut(slab1, a=(target_xy[0], 0, 0), b=(0, target_xy[1], 0), origo=(0,0,0))
    #     # cut_slab2 = cut(slab2, a=(target_xy[0], 0, 0), b=(0, target_xy[1], 0), origo=(0,0,0))
    #     position1 = slab1.get_positions()
    #     position2 = slab2.get_positions()
    #     # 条件（x>=12 または y>=12）に合う原子のインデックスを探す
    #     # np.whereは条件に合う要素のインデックスを返す
    #     indices_to_delete1 = np.where(
    #         (position1[:, 0] >= target_xy[0]) | (position1[:, 1] >= target_xy[1])
    #     )[0]
    #     indices_to_delete2 = np.where(
    #         (position2[:, 0] >= target_xy[0]) | (position2[:, 1] >= target_xy[1])
    #     )[0]
    #     # 見つかったインデックスの原子を削除
    #     cut_slab1=slab1.copy()
    #     cut_slab2=slab2.copy()
    #     del cut_slab1[indices_to_delete1]
    #     del cut_slab2[indices_to_delete2]
    #     print(f"界面から遠い原子を {len(indices_to_delete1)},{len(indices_to_delete2)}削除")
    #     # Step 2: 緩和済み表面の位置を特定
    #     z1_max = cut_slab1.positions[:, 2].max()
    #     z2_min = cut_slab2.positions[:, 2].min()

    #     # Step 3: 界面が中央に来るように各スラブをZ方向にシフト
    #     cut_slab1.positions[:, 2] += (target_z / 2) - (separation / 2) - z1_max
    #     cut_slab2.positions[:, 2] += (target_z / 2) + (separation / 2) - z2_min
        
    #     # Step 4: 2つのスラブを結合
    #     interface = cut_slab1 + cut_slab2
        
    #     # Step 5: 最終的なセルを設定 (原子はスケールしない)
    #     cell_z=target_z*2+2
    #     interface.set_cell([target_xy[0], target_xy[1], cell_z], scale_atoms=False)
    #     interface.pbc = (True, True, True) # MD用にPBCをTrueに設定
        
    #     # Step 6: セルからはみ出した原子（界面から遠い原子）を削除
    #     # np.deleteはインデックスの重複を許さないため、ユニークなインデックスを取得
        
    #     # if len(indices_to_delete) > 0:
    #     #     del interface[indices_to_delete]
    #     #     print(f"    -> 界面から遠い原子を {len(indices_to_delete)} 個削除しました。")
        
    #     # Step 7: 原子をセル内に折りたたむ
    #     interface.wrap()
    #     print(f"    -> 構築後の原子数: {len(interface)}")
    #     return interface
    def build_and_cut_interface(slab1, slab2, target_xy=(12.0, 12.0), separation=2.0, vacuum=1.0):
        # Step 1: 各スラブから指定範囲外の原子を削除
        position1 = slab1.get_positions()
        position2 = slab2.get_positions()
        indices_to_delete1 = np.where(
            (position1[:, 0] >= target_xy[0]) | (position1[:, 1] >= target_xy[1])
        )[0]
        indices_to_delete2 = np.where(
            (position2[:, 0] >= target_xy[0]) | (position2[:, 1] >= target_xy[1])
        )[0]
        cut_slab1=slab1.copy()
        cut_slab2=slab2.copy()
        if len(indices_to_delete1) > 0: del cut_slab1[indices_to_delete1]
        if len(indices_to_delete2) > 0: del cut_slab2[indices_to_delete2]
        print(f"    -> スラブ1, 2からそれぞれ {len(indices_to_delete1)}, {len(indices_to_delete2)} 個の原子を削除。")
        
        # Step 2: 界面間の距離を調整してスタック
        z1_max = cut_slab1.positions[:, 2].max()
        z2_min = cut_slab2.positions[:, 2].min()
        cut_slab2.positions[:, 2] += z1_max - z2_min + separation
        
        # Step 3: 2つのスラブを結合し、新しいセルを設定
        interface = cut_slab1 + cut_slab2
        interface.set_cell([target_xy[0], target_xy[1], 50], scale_atoms=False) # Zは仮の高さ
        
        # Step 4: Z方向の周期性を無くし、真空層を設けて中央に配置
        interface.pbc = (True, True, False)
        interface.center(vacuum=vacuum, axis=2)
        
        print(f"    -> 構築後の原子数: {len(interface)}。Z方向に真空層を持つ2層構造を作成。")
        return interface
    # def cut_slab_to_size(self, slab, target_xy=(12.0, 12.0)):
    #     print(f"    -> スラブを ({target_xy[0]}, {target_xy[1]}) Å にカット...")
    #     # 新しいセルの基底ベクトルを定義
    #     new_a = (target_xy[0], 0, 0)
    #     new_b = (0, target_xy[1], 0)
        
    #     # 元のスラブの中心を基準にカット
    #     origo_cart = slab.cell.sum(axis=0) / 2 - (target_xy[0]/2, target_xy[1]/2, 0)
    #     origo = np.linalg.solve(slab.cell.T, origo_cart.T)

    #     cut_slab = cut(slab, a=new_a, b=new_b, origo=origo)
    #     cut_slab.pbc = (True, True, False) # pbcを再設定
    #     print(f"    -> カット後の原子数: {len(cut_slab)}")
    #     return cut_slab
    def find_and_categorize_surfaces(self):
        print(f"🔍 '{self.surfaces_dir}' から最適化済み表面 (*_final.xyz) を分類中...")
        if not self.surfaces_dir.is_dir():
             print(f"🛑 エラー: 表面ディレクトリ '{self.surfaces_dir}' が見つかりません。")
             return [], []
        
        all_surfaces = list(self.surfaces_dir.glob("*_final.xyz"))
        
        al_surfaces = [p for p in all_surfaces if "Al" in p.stem]
        cathode_surfaces = [p for p in all_surfaces if "NMC" in p.stem]
        print(f"  - Al表面: {len(al_surfaces)}個,  正極材表面: {len(cathode_surfaces)}個")
        return al_surfaces, cathode_surfaces

    def create_and_optimize_interface(self, slab1_path, slab2_path):
        slab1_name = slab1_path.stem.replace('_pos_opt_final', '').replace('_cell_opt_final', '')
        slab2_name = slab2_path.stem.replace('_pos_opt_final', '').replace('_cell_opt_final', '')
        interface_name = f"Interface_{slab1_name}_on_{slab2_name}"
        output_path = self.interfaces_dir / f"{interface_name}.cif"
        
        if output_path.exists():
            print(f"✅ 既存ファイルあり、スキップ: {output_path.name}")
            return

        print(f"\n--- 界面を構築・最適化: {slab1_name} + {slab2_name} ---")
        try:
            slab1, slab2 = read(slab1_path), read(slab2_path)
            # 修正された関数を呼び出す
            interface = self.build_and_cut_interface(slab1, slab2)
            traj_path = self.interfaces_dir / f"{interface_name}.traj"
            if run_matlantis_optimization(interface, traj_path, fmax=0.05, name=interface_name):
                # 最適化成功時にCIFファイルを書き込む
                write(str(output_path), interface)
                print(f"  -> 💾 保存しました: {output_path.name}")
        # 'except'のインデントを'try'に合わせる
        except Exception as e:
            print(f"  -> ❌ この界面の作成中にエラーが発生したため、スキップします: {e}")
    def run(self):
        al_slabs, cathode_slabs = self.find_and_categorize_surfaces()
        if not al_slabs and not cathode_slabs:
            print(" -> 処理対象の表面ファイルがないため、終了します。")
            return
            
        print("\n--- カテゴリ1: Alを含む界面 ---")
        if al_slabs and cathode_slabs:
            for al_path in al_slabs:
                for cathode_path in cathode_slabs:
                    self.create_and_optimize_interface(al_path, cathode_path)
        else:
            print(" -> スキップ (Al表面または正極材表面のいずれかが不足)")
            
        # print("\n--- カテゴリ2: Alを含まない界面 ---")
        # if len(cathode_slabs) >= 2:
        #     for pair in itertools.combinations(cathode_slabs, 2):
        #         self.create_and_optimize_interface(pair[0], pair[1])
        # else:
        #     print(" -> スキップ (正極材表面が2つ未満)")
        print("\n✨ STEP 2 完了。")

def show_results_viewer(results_dir):
    print(f"\n--- 🔬 結果ビューワー: '{results_dir}' ---")
    files = sorted(list(Path(results_dir).glob("*.cif")))
    if not files:
        print("表示できる構造がありません。")
        return
    options = {"-- 選択してください --": None}
    options.update({p.name: p for p in files})
    widgets.interact(
        lambda file_path: display(nv.show_ase(read(file_path)) if file_path else None),
        file_path=widgets.Dropdown(options=options, description='Interface:', layout={'width': 'max-content'})
    )

# --- メイン実行部 ---
if __name__ == "__main__":
    # ===== 設定項目 =====
    # 1. Trajectoryファイルが保存されているディレクトリ
    OPTIMIZATION_OUTPUT_DIR = "/home/jovyan/Kaori/MD/LiB_2/structure/output"
    
    # 2. 最適化した界面の保存先
    INTERFACES_DIR = "/home/jovyan/Kaori/MD/LiB_2/structure/output/optimized_interfaces"
    # ====================

    clear_output(wait=True)
    
    # STEP 1: メモリ管理 (traj -> xyz変換 & 削除)
    cleanup = TrajectoryCleanup(OPTIMIZATION_OUTPUT_DIR)
    cleanup.run()
    
    # STEP 2: 界面の構築と最適化
    interface_opt = InterfaceOptimizer(surfaces_dir=OPTIMIZATION_OUTPUT_DIR, interfaces_dir=INTERFACES_DIR)
    interface_opt.run()
    
    print("\n\n✨ 全てのワークフローが完了しました。 ✨")
    
    # STEP 3: 結果の表示
    show_results_viewer(INTERFACES_DIR)


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 228)

In [ ]:
#!/usr/bin/env python3
"""
Trajectoryファイルの整理と、それに基づく界面構築・最適化を自動化するプログラム。

[エラー耐性強化版]
- 破損したファイルなど、特定ファイルの処理でエラーが発生しても全体が停止せず、
  次のファイルの処理を続行します。

STEP 1: Trajectoryファイルの分析とクリーンアップ
  - 0KBの.trajファイル（失敗した計算）を削除
  - 正常な.trajファイルを初期/最終構造の.xyzに変換し、元ファイルを削除

STEP 2: 界面の構築と最適化
  - STEP 1で生成された最終構造(.xyz)を用いて界面を構築
  - Matlantisを用いて原子位置のみを最適化
"""
import os
import itertools
import numpy as np
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display, clear_output
import nglview as nv

# ASE
from ase.io import read, write
from ase.build import stack,cut

# Matlantis / PFP
from matlantis_features.atoms import MatlantisAtoms
from matlantis_features.features.common.opt import FireLBFGSASEOptFeature
from matlantis_features.utils.calculators import pfp_estimator_fn
from pfp_api_client.pfp.estimator import EstimatorCalcMode

# --- STEP 1: Trajectoryファイルの整理クラス ---
class TrajectoryCleanup:
    """
    .trajファイルを分析し、.xyzに変換してクリーンアップするクラス。
    """
    def __init__(self, directory):
        self.directory = Path(directory)
        print("--- STEP 1: Trajectoryファイルの分析と整理を開始 ---")
        if not self.directory.is_dir():
            print(f"🛑 エラー: ディレクトリ '{self.directory}' が見つかりません。")
            self.files_to_process = []
        else:
            self.files_to_process = list(self.directory.glob("*.traj"))
            print(f"🔍 '{self.directory}' から {len(self.files_to_process)} 個の .traj ファイルを発見。")

    def process_file(self, traj_path):
        """
        1つの.trajファイルを処理する。
        [修正] この関数全体をtry...exceptで囲み、エラー耐性を向上。
        """
        try:
            # ファイルサイズを確認
            file_size = traj_path.stat().st_size
            if file_size == 0:
                print(f"  -> 🗑️  0KBファイルを削除します: {traj_path.name}")
                os.remove(traj_path)
                return

            # 正常なファイルは変換
            base_name = traj_path.stem
            initial_xyz_path = self.directory / f"{base_name}_initial.xyz"
            final_xyz_path = self.directory / f"{base_name}_final.xyz"

            print(f"  -> 🔄 変換中: {traj_path.name}")

            # Trajectoryから初期構造と最終構造を読み込み
            atoms_list = read(traj_path, index=":")
            if not atoms_list:
                print(f"  -> ❌ 読み込みエラー（空のTrajectory）、ファイルを削除します: {traj_path.name}")
                os.remove(traj_path)
                return

            # .xyzファイルとして保存
            write(str(initial_xyz_path), atoms_list[0])
            write(str(final_xyz_path), atoms_list[-1])

            # 元の.trajファイルを削除
            os.remove(traj_path)
            print(f"  -> ✔️ 変換成功、元のファイルを削除しました: {traj_path.name}")

        except Exception as e:
            # 予期せぬエラーが発生した場合、ログを出力して処理を続行
            print(f"  -> ❌ 重大なエラーが発生したため、このファイルはスキップします ({traj_path.name}): {e}")

    def run(self):
        """全ての対象ファイルに対して処理を実行する。"""
        if not self.files_to_process:
            print(" -> 処理対象の.trajファイルがないため、スキップします。")
            return
        for traj_file in self.files_to_process:
            self.process_file(traj_file)
        print("\n✨ STEP 1 完了。")


# --- Matlantis最適化関数 (共通) ---
def run_matlantis_optimization(atoms, trajectory_path, fmax=0.05, name="structure"):
    print(f"  -> Matlantis最適化開始 ({name}, fmax = {fmax}) ...")
    matlantis_atoms = MatlantisAtoms(atoms)
    estimator_function = pfp_estimator_fn(
        model_version='v7.0.0', calc_mode=EstimatorCalcMode.CRYSTAL_PLUS_D3
    )
    position_optimizer = FireLBFGSASEOptFeature(
        estimator_fn=estimator_function, filter=False, trajectory=str(trajectory_path),
        n_run=5000, fmax=fmax, show_progress_bar=True
    )
    try:
        result = position_optimizer(matlantis_atoms)
        optimized_atoms = result.atoms.ase_atoms
        final_energy = result.output.energy_log[-1]
        print(f"  -> ✔️ 最適化完了！ エネルギー: {final_energy:.3f} eV")
        xyz_save_path = Path(trajectory_path).with_suffix('.xyz')
        print(f"  -> 💾 最適化後の構造を保存します: {xyz_save_path.name}")
        write(str(xyz_save_path), optimized_atoms)
        return True
    except Exception as e:
        print(f"  -> ❌ 最適化中にエラーが発生: {e}")
        return False

# --- STEP 2: 界面最適化クラス ---
class InterfaceOptimizer:
    def __init__(self, surfaces_dir, interfaces_dir):
        self.surfaces_dir = Path(surfaces_dir)
        self.interfaces_dir = Path(interfaces_dir)
        self.interfaces_dir.mkdir(parents=True, exist_ok=True)
        print(f"\n--- STEP 2: 界面構築・最適化プロセスを開始 ---")
        print(f"✅ 界面構造の保存先: '{self.interfaces_dir}'")

    def build_and_cut_interface(self, slab1, slab2, target_xy=(12.0, 12.0), separation=2.0, vacuum=2.0):
        """
        緩和済み表面を維持しつつ、指定サイズで2層の界面を構築する。
        """
        print(f"    -> 2層界面を ({target_xy[0]}, {target_xy[1]}) Å に構築...")

        # Step 1: 各スラブから指定範囲外の原子を削除
        position1 = slab1.get_positions()
        position2 = slab2.get_positions()
        indices_to_delete1 = np.where(
            (position1[:, 0] >= target_xy[0]) | (position1[:, 1] >= target_xy[1])
        )[0]
        indices_to_delete2 = np.where(
            (position2[:, 0] >= target_xy[0]) | (position2[:, 1] >= target_xy[1])
        )[0]
        cut_slab1=slab1.copy()
        cut_slab2=slab2.copy()
        if len(indices_to_delete1) > 0: del cut_slab1[indices_to_delete1]
        if len(indices_to_delete2) > 0: del cut_slab2[indices_to_delete2]
        print(f"    -> スラブ1, 2からそれぞれ {len(indices_to_delete1)}, {len(indices_to_delete2)} 個の原子を削除。")

        # Step 2: 界面間の距離を調整してスタック
        z1_max = cut_slab1.positions[:, 2].max()
        z2_min = cut_slab2.positions[:, 2].min()
        cut_slab2.positions[:, 2] += z1_max - z2_min + separation

        # Step 3: 2つのスラブを結合し、新しいセルを設定
        interface = cut_slab1 + cut_slab2
        interface.set_cell([target_xy[0], target_xy[1], 50], scale_atoms=False) # Zは仮の高さ

        # Step 4: Z方向の周期性を無くし、真空層を設けて中央に配置
        interface.pbc = (True, True, False)
        interface.center(vacuum=vacuum, axis=2)

        print(f"    -> 構築後の原子数: {len(interface)}。Z方向に真空層を持つ2層構造を作成。")
        return interface

    def cut_slab_to_size(self, slab, target_xy=(12.0, 12.0)):
        print(f"    -> スラブを ({target_xy[0]}, {target_xy[1]}) Å にカット...")
        # 新しいセルの基底ベクトルを定義
        new_a = (target_xy[0], 0, 0)
        new_b = (0, target_xy[1], 0)

        # 元のスラブの中心を基準にカット
        origo_cart = slab.cell.sum(axis=0) / 2 - (target_xy[0]/2, target_xy[1]/2, 0)
        origo = np.linalg.solve(slab.cell.T, origo_cart.T)

        cut_slab = cut(slab, a=new_a, b=new_b, origo=origo)
        cut_slab.pbc = (True, True, False) # pbcを再設定
        print(f"    -> カット後の原子数: {len(cut_slab)}")
        return cut_slab

    def find_and_categorize_surfaces(self):
        print(f"🔍 '{self.surfaces_dir}' から最適化済み表面 (*_final.xyz) を分類中...")
        if not self.surfaces_dir.is_dir():
            print(f"🛑 エラー: 表面ディレクトリ '{self.surfaces_dir}' が見つかりません。")
            return [], []

        all_surfaces = list(self.surfaces_dir.glob("*_final.xyz"))

        al_surfaces = [p for p in all_surfaces if "Al" in p.stem]
        cathode_surfaces = [p for p in all_surfaces if "NMC" in p.stem]
        print(f"  - Al表面: {len(al_surfaces)}個,  正極材表面: {len(cathode_surfaces)}個")
        return al_surfaces, cathode_surfaces

    def create_and_optimize_interface(self, slab1_path, slab2_path):
        slab1_name = slab1_path.stem.replace('_pos_opt_final', '').replace('_cell_opt_final', '')
        slab2_name = slab2_path.stem.replace('_pos_opt_final', '').replace('_cell_opt_final', '')
        interface_name = f"Interface_{slab1_name}_on_{slab2_name}"
        output_path = self.interfaces_dir / f"{interface_name}.cif"

        if output_path.exists():
            print(f"✅ 既存ファイルあり、スキップ: {output_path.name}")
            return

        print(f"\n--- 界面を構築・最適化: {slab1_name} + {slab2_name} ---")
        try:
            slab1, slab2 = read(slab1_path), read(slab2_path)
            # 修正された関数を呼び出す
            interface = self.build_and_cut_interface(slab1, slab2)
            traj_path = self.interfaces_dir / f"{interface_name}.traj"
            if run_matlantis_optimization(interface, traj_path, fmax=0.05, name=interface_name):
                # 最適化成功時にCIFファイルを書き込む
                write(str(output_path), interface)
                print(f"  -> 💾 保存しました: {output_path.name}")
        # 'except'のインデントを'try'に合わせる
        except Exception as e:
            print(f"  -> ❌ この界面の作成中にエラーが発生したため、スキップします: {e}")

    def run(self):
        al_slabs, cathode_slabs = self.find_and_categorize_surfaces()
        if not al_slabs and not cathode_slabs:
            print(" -> 処理対象の表面ファイルがないため、終了します。")
            return

        print("\n--- カテゴリ1: Alを含む界面 ---")
        if al_slabs and cathode_slabs:
            for al_path in al_slabs:
                for cathode_path in cathode_slabs:
                    self.create_and_optimize_interface(al_path, cathode_path)
        else:
            print(" -> スキップ (Al表面または正極材表面のいずれかが不足)")

        # print("\n--- カテゴリ2: Alを含まない界面 ---")
        # if len(cathode_slabs) >= 2:
        #     for pair in itertools.combinations(cathode_slabs, 2):
        #         self.create_and_optimize_interface(pair[0], pair[1])
        # else:
        #     print(" -> スキップ (正極材表面が2つ未満)")
        print("\n✨ STEP 2 完了。")


        
def show_results_viewer(results_dir):
    print(f"\n--- 🔬 結果ビューワー: '{results_dir}' ---")
    files = sorted(list(Path(results_dir).glob("*.cif")))
    if not files:
        print("表示できる構造がありません。")
        return
    options = {"-- 選択してください --": None}
    options.update({p.name: p for p in files})
    widgets.interact(
        lambda file_path: display(nv.show_ase(read(file_path)) if file_path else None),
        file_path=widgets.Dropdown(options=options, description='Interface:', layout={'width': 'max-content'})
    )

# --- メイン実行部 ---
if __name__ == "__main__":
    # ===== 設定項目 =====
    # 1. Trajectoryファイルが保存されているディレクトリ
    OPTIMIZATION_OUTPUT_DIR = "/home/jovyan/Kaori/MD/LiB_2/structure/output"

    # 2. 最適化した界面の保存先
    INTERFACES_DIR = "/home/jovyan/Kaori/MD/LiB_2/structure/output/optimized_interfaces"
    # ====================

    clear_output(wait=True)

    # STEP 1: メモリ管理 (traj -> xyz変換 & 削除)
    cleanup = TrajectoryCleanup(OPTIMIZATION_OUTPUT_DIR)
    cleanup.run()

    # STEP 2: 界面の構築と最適化
    interface_opt = InterfaceOptimizer(surfaces_dir=OPTIMIZATION_OUTPUT_DIR, interfaces_dir=INTERFACES_DIR)
    interface_opt.run()

    print("\n\n✨ 全てのワークフローが完了しました。 ✨")

    # STEP 3: 結果の表示
    show_results_viewer(INTERFACES_DIR)

In [3]:
#!/usr/bin/env python3
"""
Trajectoryファイルの整理と、それに基づく界面構築・最適化を自動化するプログラム。

[エラー耐性強化版]
- 破損したファイルなど、特定ファイルの処理でエラーが発生しても全体が停止せず、
  次のファイルの処理を続行します。

STEP 1: Trajectoryファイルの分析とクリーンアップ
  - 0KBの.trajファイル（失敗した計算）を削除
  - 正常な.trajファイルを初期/最終構造の.xyzに変換し、元ファイルを削除

STEP 2: 界面の構築と最適化
  - STEP 1で生成された最終構造(.xyz)を用いて界面を構築
  - Matlantisを用いて原子位置のみを最適化
"""
import os
import itertools
import numpy as np
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display, clear_output
import nglview as nv

# ASE
from ase.io import read, write
from ase.build import stack,cut

# Matlantis / PFP
from matlantis_features.atoms import MatlantisAtoms
from matlantis_features.features.common.opt import FireLBFGSASEOptFeature
from matlantis_features.utils.calculators import pfp_estimator_fn
from pfp_api_client.pfp.estimator import EstimatorCalcMode

# --- STEP 1: Trajectoryファイルの整理クラス ---
class TrajectoryCleanup:
    """
    .trajファイルを分析し、.xyzに変換してクリーンアップするクラス。
    """
    def __init__(self, directory):
        self.directory = Path(directory)
        print("--- STEP 1: Trajectoryファイルの分析と整理を開始 ---")
        if not self.directory.is_dir():
            print(f"🛑 エラー: ディレクトリ '{self.directory}' が見つかりません。")
            self.files_to_process = []
        else:
            self.files_to_process = list(self.directory.glob("*.traj"))
            print(f"🔍 '{self.directory}' から {len(self.files_to_process)} 個の .traj ファイルを発見。")

    def process_file(self, traj_path):
        """
        1つの.trajファイルを処理する。
        [修正] この関数全体をtry...exceptで囲み、エラー耐性を向上。
        """
        try:
            # ファイルサイズを確認
            file_size = traj_path.stat().st_size
            if file_size == 0:
                print(f"  -> 🗑️  0KBファイルを削除します: {traj_path.name}")
                os.remove(traj_path)
                return

            # 正常なファイルは変換
            base_name = traj_path.stem
            initial_xyz_path = self.directory / f"{base_name}_initial.xyz"
            final_xyz_path = self.directory / f"{base_name}_final.xyz"
            
            print(f"  -> 🔄 変換中: {traj_path.name}")
            
            # Trajectoryから初期構造と最終構造を読み込み
            atoms_list = read(traj_path, index=":")
            if not atoms_list:
                print(f"  -> ❌ 読み込みエラー（空のTrajectory）、ファイルを削除します: {traj_path.name}")
                os.remove(traj_path)
                return

            # .xyzファイルとして保存
            write(str(initial_xyz_path), atoms_list[0])
            write(str(final_xyz_path), atoms_list[-1])
            
            # 元の.trajファイルを削除
            os.remove(traj_path)
            print(f"  -> ✔️ 変換成功、元のファイルを削除しました: {traj_path.name}")

        except Exception as e:
            # 予期せぬエラーが発生した場合、ログを出力して処理を続行
            print(f"  -> ❌ 重大なエラーが発生したため、このファイルはスキップします ({traj_path.name}): {e}")

    def run(self):
        """全ての対象ファイルに対して処理を実行する。"""
        if not self.files_to_process:
            print(" -> 処理対象の.trajファイルがないため、スキップします。")
            return
        for traj_file in self.files_to_process:
            self.process_file(traj_file)
        print("\n✨ STEP 1 完了。")


# --- Matlantis最適化関数 (共通) ---
def run_matlantis_optimization(atoms, trajectory_path, fmax=0.05, name="structure"):
    print(f"  -> Matlantis最適化開始 ({name}, fmax = {fmax}) ...")
    matlantis_atoms = MatlantisAtoms(atoms)
    estimator_function = pfp_estimator_fn(
        model_version='v7.0.0', calc_mode=EstimatorCalcMode.CRYSTAL_PLUS_D3
    )
    position_optimizer = FireLBFGSASEOptFeature(
        estimator_fn=estimator_function, filter=False, trajectory=str(trajectory_path),
        n_run=5000, fmax=fmax, show_progress_bar=True
    )
    try:
        result = position_optimizer(matlantis_atoms)
        optimized_atoms = result.atoms.ase_atoms
        final_energy = result.output.energy_log[-1]
        print(f"  -> ✔️ 最適化完了！ エネルギー: {final_energy:.3f} eV")
        xyz_save_path = Path(trajectory_path).with_suffix('.xyz')
        print(f"  -> 💾 最適化後の構造を保存します: {xyz_save_path.name}")
        write(str(xyz_save_path), optimized_atoms)
        return True
    except Exception as e:
        print(f"  -> ❌ 最適化中にエラーが発生: {e}")
        return False

# --- STEP 2: 界面最適化クラス ---
class InterfaceOptimizer:
    def __init__(self, surfaces_dir, interfaces_dir):
        self.surfaces_dir = Path(surfaces_dir)
        self.interfaces_dir = Path(interfaces_dir)
        self.interfaces_dir.mkdir(parents=True, exist_ok=True)
        print(f"\n--- STEP 2: 界面構築・最適化プロセスを開始 ---")
        print(f"✅ 界面構造の保存先: '{self.interfaces_dir}'")

    # def build_and_cut_interface(self, slab1, slab2, target_xy=(12.0, 12.0), target_z=20.0, separation=2.0):
    #     """
    #     緩和済み表面を維持しつつ、指定サイズに界面を精密に構築・カットする。
    #     """
    #     print(f"    -> 界面を ({target_xy[0]}, {target_xy[1]}, {target_z}) Å に精密構築...")
    #     # Step 1: 各スラブをX,Y方向にカット
    #     # cut_slab1 = cut(slab1, a=(target_xy[0], 0, 0), b=(0, target_xy[1], 0), origo=(0,0,0))
    #     # cut_slab2 = cut(slab2, a=(target_xy[0], 0, 0), b=(0, target_xy[1], 0), origo=(0,0,0))
    #     position1 = slab1.get_positions()
    #     position2 = slab2.get_positions()
    #     # 条件（x>=12 または y>=12）に合う原子のインデックスを探す
    #     # np.whereは条件に合う要素のインデックスを返す
    #     indices_to_delete1 = np.where(
    #         (position1[:, 0] >= target_xy[0]) | (position1[:, 1] >= target_xy[1])
    #     )[0]
    #     indices_to_delete2 = np.where(
    #         (position2[:, 0] >= target_xy[0]) | (position2[:, 1] >= target_xy[1])
    #     )[0]
    #     # 見つかったインデックスの原子を削除
    #     cut_slab1=slab1.copy()
    #     cut_slab2=slab2.copy()
    #     del cut_slab1[indices_to_delete1]
    #     del cut_slab2[indices_to_delete2]
    #     print(f"界面から遠い原子を {len(indices_to_delete1)},{len(indices_to_delete2)}削除")
    #     # Step 2: 緩和済み表面の位置を特定
    #     z1_max = cut_slab1.positions[:, 2].max()
    #     z2_min = cut_slab2.positions[:, 2].min()

    #     # Step 3: 界面が中央に来るように各スラブをZ方向にシフト
    #     cut_slab1.positions[:, 2] += (target_z / 2) - (separation / 2) - z1_max
    #     cut_slab2.positions[:, 2] += (target_z / 2) + (separation / 2) - z2_min
        
    #     # Step 4: 2つのスラブを結合
    #     interface = cut_slab1 + cut_slab2
        
    #     # Step 5: 最終的なセルを設定 (原子はスケールしない)
    #     cell_z=target_z*2+2
    #     interface.set_cell([target_xy[0], target_xy[1], cell_z], scale_atoms=False)
    #     interface.pbc = (True, True, True) # MD用にPBCをTrueに設定
        
    #     # Step 6: セルからはみ出した原子（界面から遠い原子）を削除
    #     # np.deleteはインデックスの重複を許さないため、ユニークなインデックスを取得
        
    #     # if len(indices_to_delete) > 0:
    #     #     del interface[indices_to_delete]
    #     #     print(f"    -> 界面から遠い原子を {len(indices_to_delete)} 個削除しました。")
        
    #     # Step 7: 原子をセル内に折りたたむ
    #     interface.wrap()
    #     print(f"    -> 構築後の原子数: {len(interface)}")
    #     return interface
    def build_and_cut_interface(self, slab1, slab2, target_xy=(12.0, 12.0), separation=2.0, vacuum=1.0):
        """
        緩和済み表面を維持しつつ、指定サイズで2層の界面を構築する。
        """
        print(f"    -> 2層界面を ({target_xy[0]}, {target_xy[1]}) Å に構築...")
        
        # Step 1: 各スラブから指定範囲外の原子を削除
        position1 = slab1.get_positions()
        position2 = slab2.get_positions()
        indices_to_delete1 = np.where(
            (position1[:, 0] >= target_xy[0]) | (position1[:, 1] >= target_xy[1])
        )[0]
        indices_to_delete2 = np.where(
            (position2[:, 0] >= target_xy[0]) | (position2[:, 1] >= target_xy[1])
        )[0]
        cut_slab1=slab1.copy()
        cut_slab2=slab2.copy()
        if len(indices_to_delete1) > 0: del cut_slab1[indices_to_delete1]
        if len(indices_to_delete2) > 0: del cut_slab2[indices_to_delete2]
        print(f"    -> スラブ1, 2からそれぞれ {len(indices_to_delete1)}, {len(indices_to_delete2)} 個の原子を削除。")
        
        # Step 2: 界面間の距離を調整してスタック
        z1_max = cut_slab1.positions[:, 2].max()
        z2_min = cut_slab2.positions[:, 2].min()
        cut_slab2.positions[:, 2] += z1_max - z2_min + separation
        
        # Step 3: 2つのスラブを結合し、新しいセルを設定
        interface = cut_slab1 + cut_slab2
        interface.set_cell([target_xy[0], target_xy[1], 50], scale_atoms=False) # Zは仮の高さ
        
        # Step 4: Z方向の周期性を無くし、真空層を設けて中央に配置
        interface.pbc = (True, True, False)
        interface.center(vacuum=vacuum, axis=2)
        
        print(f"    -> 構築後の原子数: {len(interface)}。Z方向に真空層を持つ2層構造を作成。")
        return interface
    def cut_slab_to_size(self, slab, target_xy=(12.0, 12.0)):
        """
        ASEのcut機能を使って、スラブを目標のx,yサイズに切り出す。
        """
        print(f"    -> スラブを ({target_xy[0]}, {target_xy[1]}) Å にカット...")
        # 新しいセルの基底ベクトルを定義
        new_a = (target_xy[0], 0, 0)
        new_b = (0, target_xy[1], 0)
        
        # 元のスラブの中心を基準にカット
        origo_cart = slab.cell.sum(axis=0) / 2 - (target_xy[0]/2, target_xy[1]/2, 0)
        origo = np.linalg.solve(slab.cell.T, origo_cart.T)

        cut_slab = cut(slab, a=new_a, b=new_b, origo=origo)
        cut_slab.pbc = (True, True, False) # pbcを再設定
        print(f"    -> カット後の原子数: {len(cut_slab)}")
        return cut_slab

    def find_and_categorize_surfaces(self):
        print(f"🔍 '{self.surfaces_dir}' から最適化済み表面 (*_final.xyz) を分類中...")
        if not self.surfaces_dir.is_dir():
             print(f"🛑 エラー: 表面ディレクトリ '{self.surfaces_dir}' が見つかりません。")
             return [], []
        
        all_surfaces = list(self.surfaces_dir.glob("*_final.xyz"))
        
        al_surfaces = [p for p in all_surfaces if "Al" in p.stem]
        cathode_surfaces = [p for p in all_surfaces if "NMC" in p.stem]
        print(f"  - Al表面: {len(al_surfaces)}個,  正極材表面: {len(cathode_surfaces)}個")
        return al_surfaces, cathode_surfaces

    def create_and_optimize_interface(self, slab1_path, slab2_path):
        slab1_name = slab1_path.stem.replace('_pos_opt_final', '').replace('_cell_opt_final', '')
        slab2_name = slab2_path.stem.replace('_pos_opt_final', '').replace('_cell_opt_final', '')
        interface_name = f"Interface_{slab1_name}_on_{slab2_name}"
        output_path = self.interfaces_dir / f"{interface_name}.cif"
        
        if output_path.exists():
            print(f"✅ 既存ファイルあり、スキップ: {output_path.name}")
            return

        print(f"\n--- 界面を構築・最適化: {slab1_name} + {slab2_name} ---")
        try:
            slab1, slab2 = read(slab1_path), read(slab2_path)
            # 修正された関数を呼び出す
            interface = self.build_and_cut_interface(slab1, slab2) 
            traj_path = self.interfaces_dir / f"{interface_name}.traj"
            if run_matlantis_optimization(interface, traj_path, fmax=0.05, name=interface_name):
                write(str(output_path), interface)
                print(f"  -> 💾 保存しました: {output_path.name}")
        except Exception as e:
            print(f"  -> ❌ この界面の作成中にエラーが発生したため、スキップします: {e}")

    def run(self):
        al_slabs, cathode_slabs = self.find_and_categorize_surfaces()
        if not al_slabs and not cathode_slabs:
            print(" -> 処理対象の表面ファイルがないため、終了します。")
            return
            
        print("\n--- カテゴリ1: Alを含む界面 ---")
        if al_slabs and cathode_slabs:
            for al_path in al_slabs:
                for cathode_path in cathode_slabs:
                    self.create_and_optimize_interface(al_path, cathode_path)
        else:
            print(" -> スキップ (Al表面または正極材表面のいずれかが不足)")
            
        # print("\n--- カテゴリ2: Alを含まない界面 ---")
        # if len(cathode_slabs) >= 2:
        #     for pair in itertools.combinations(cathode_slabs, 2):
        #         self.create_and_optimize_interface(pair[0], pair[1])
        # else:
        #     print(" -> スキップ (正極材表面が2つ未満)")
        print("\n✨ STEP 2 完了。")

def show_results_viewer(results_dir):
    print(f"\n--- 🔬 結果ビューワー: '{results_dir}' ---")
    files = sorted(list(Path(results_dir).glob("*.cif")))
    if not files:
        print("表示できる構造がありません。")
        return
    options = {"-- 選択してください --": None}
    options.update({p.name: p for p in files})
    widgets.interact(
        lambda file_path: display(nv.show_ase(read(file_path)) if file_path else None),
        file_path=widgets.Dropdown(options=options, description='Interface:', layout={'width': 'max-content'})
    )

# --- メイン実行部 ---
if __name__ == "__main__":
    # ===== 設定項目 =====
    # 1. Trajectoryファイルが保存されているディレクトリ
    OPTIMIZATION_OUTPUT_DIR = "/home/jovyan/Kaori/MD/LiB_2/structure/output"
    
    # 2. 最適化した界面の保存先
    INTERFACES_DIR = "/home/jovyan/Kaori/MD/LiB_2/structure/output/optimized_interfaces"
    # ====================

    clear_output(wait=True)
    
    # STEP 1: メモリ管理 (traj -> xyz変換 & 削除)
    cleanup = TrajectoryCleanup(OPTIMIZATION_OUTPUT_DIR)
    cleanup.run()
    
    # STEP 2: 界面の構築と最適化
    interface_opt = InterfaceOptimizer(surfaces_dir=OPTIMIZATION_OUTPUT_DIR, interfaces_dir=INTERFACES_DIR)
    interface_opt.run()
    
    print("\n\n✨ 全てのワークフローが完了しました。 ✨")
    
    # STEP 3: 結果の表示
    show_results_viewer(INTERFACES_DIR)


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 219)

In [4]:
from ase.io import read, write,Trajectory
# from pfcc.extras import Trajectory
atoms=Trajectory("/home/jovyan/Kaori/MD/LiB_2/structure/output/Al2O3_10-12_(1, 0, 2)_cell_opt.traj")[-1]
ini=Trajectory("/home/jovyan/Kaori/MD/LiB_2/structure/output/Al2O3_10-12_(1, 0, 2)_cell_opt.traj")[0]
write("/home/jovyan/Kaori/MD/LiB_2/structure/output/Al2O3_10-12_(1, 0, 2)_cell_opt_initial.xyz", ini)

write("/home/jovyan/Kaori/MD/LiB_2/structure/output/Al2O3_10-12_(1, 0, 2)_cell_opt_final.xyz", atoms)

In [1]:
#!/usr/bin/env python3
"""
NMC系材料とAl界面の体系的構造構築・最適化プログラム
リチウムイオン電池リサイクル研究用

Features:
- 段階的構造最適化
- 組成変動NMC構造の自動生成
- 体系的な表面・界面構築
- 複数ミラー指数対応
- カレンダリング圧力シミュレーション対応
"""
# 汎用モジュール
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from IPython.display import display_png
from IPython.display import Image as ImageWidget
import ipywidgets as widgets
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.widgets import Slider
from matplotlib.animation import PillowWriter
import seaborn as sns
import math
import optuna
import nglview as nv
import os,sys,csv,glob,shutil,re,time
from pathlib import Path
from PIL import Image, ImageDraw

# sklearn
from sklearn.metrics import mean_absolute_error


# ASE
import ase
from ase import Atoms, units
from ase.units import Bohr,Rydberg,kJ,kB,fs,Hartree,mol,kcal
from ase.io import read, write
from ase.build import surface, molecule, add_adsorbate
from ase.cluster.cubic import FaceCenteredCubic
from ase.constraints import FixAtoms, FixedPlane, FixBondLength, ExpCellFilter
# from ase.neb import SingleCalculatorNEB
from ase.neb import NEB
from ase.vibrations import Vibrations
from ase.visualize import view
from ase.optimize import QuasiNewton
from ase.thermochemistry import IdealGasThermo
from ase.build.rotate import minimize_rotation_and_translation
from ase.visualize import view
from ase.optimize import BFGS, LBFGS, FIRE
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution, Stationary
from ase.md.verlet import VelocityVerlet
from ase.md.langevin import Langevin
from ase.md.nptberendsen import NPTBerendsen, Inhomogeneous_NPTBerendsen
from ase.md import MDLogger
from ase.io import read, write, Trajectory
# from ase.calculators.dftd3 import DFTD3
from ase.build import sort

from sella import Sella, Constraints
from torch_dftd.torch_dftd3_calculator import TorchDFTD3Calculator

# PFP
from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
from pfp_api_client.pfp.estimator import Estimator
from pfp_api_client.pfp.estimator import EstimatorCalcMode
# calculator = pfp_estimator_fn(model_version='v7.0.0', calc_mode='CRYSTAL_U0_PLUS_D3')
estimator = Estimator(calc_mode='CRYSTAL_U0_PLUS_D3')
calculator = ASECalculator(estimator)
# warnings.filterwarnings('ignore')

class NMCInterfaceBuilder:
    """NMC-Al界面構造構築クラス"""
    
    def __init__(self, base_dir="/home/jovyan/Kaori/MD"):
        self.base_dir = base_dir
        self.input_dir = os.path.join(base_dir, "input")
        self.output_dir = os.path.join(base_dir, "output")
        os.makedirs(self.output_dir, exist_ok=True)
        
        # 計算器設定
        self.calculator = calculator
        
        # Miller指数リスト（重要度順、NMC111最頻面を反映）
        self.miller_indices = [
            (0, 0, 1),  # NMC(001) - 最安定面、プレート形状
            (1, 0, 4),  # NMC(104) - 中活性、多面体形状
            (0, 1, 2),  # NMC(012) - 高活性、切頭八面体形状
            (0, 1, 0),  # NMC(010) - Li拡散活性面
            (1, 1, 1),  # Al(111) - Al最安定面
            (1, 0, 0),  # Al(100), NMC(100) - 基本面
            (1, 1, 0),  # Al(110), NMC(110) - 副次面
        ]
        
        # カレンダリング圧力設定（MPa）
        self.calendering_pressures = [10, 30, 60, 120, 200]  # 実用範囲
        
    def optimize_structure_systematic(self, atoms, name="structure"):
        """段階的構造最適化"""
        print(f"=== {name}の構造最適化開始 ===")
        
        atoms.calc = self.calculator
        
        # Step 1: 格子定数最適化
        print("Step 1: 格子定数最適化")
        cell_opt = BFGS(UnitCellFilter(atoms), 
                       trajectory=f"{self.output_dir}/{name}_cell_opt.traj")
        try:
            cell_opt.run(fmax=0.01)
            print(f"格子最適化完了: {atoms.cell.cellpar()}")
        except Exception as e:
            print(f"格子最適化警告: {e}")
        
        # Step 2: 原子位置最適化
        print("Step 2: 原子位置最適化")
        pos_opt = BFGS(atoms, 
                      trajectory=f"{self.output_dir}/{name}_pos_opt.traj")
        try:
            pos_opt.run(fmax=0.005)
            print(f"位置最適化完了: エネルギー = {atoms.get_potential_energy():.3f} eV")
        except Exception as e:
            print(f"位置最適化警告: {e}")
        
        return atoms
    
    def makesurface_advanced(self, atoms, miller_indices=(1,0,0), layers=4, 
                           rep=[4,4,1], vacuum=15.0, fix_ratio=0.3, name="surface"):
        """改良版表面作成関数"""
        print(f"表面作成: {name} {miller_indices}面, {layers}層")
        
        # 表面作成
        slab = surface(atoms, miller_indices, layers, vacuum=vacuum)
        slab = slab.repeat(rep)
        
        # 位置調整
        positions = slab.get_positions()
        z_min = positions[:, 2].min()
        slab.set_positions(positions - [0, 0, z_min])
        
        # 下層原子の固定（fix_ratio分を固定）
        z_coords = slab.positions[:, 2]
        z_max = z_coords.max()
        z_threshold = z_coords.min() + (z_max - z_coords.min()) * fix_ratio
        fixed_indices = [i for i, z in enumerate(z_coords) if z <= z_threshold]
        
        if fixed_indices:
            constraint = FixAtoms(indices=fixed_indices)
            slab.set_constraint(constraint)
            print(f"固定原子数: {len(fixed_indices)}/{len(slab)}")
        
        # 構造最適化
        slab = self.optimize_structure_systematic(slab, f"{name}_{miller_indices}")
        
        return slab
    
    def create_composition_variants(self, base_atoms, base_name="NMC"):
        """組成変動NMC構造の作成（化学量論保持置換）"""
        print(f"=== {base_name}の組成変動構造作成 ===")
        
        variants = {f"{base_name}_pristine": base_atoms.copy()}
        
        # 元素置換パターン（化学量論保持）
        substitution_patterns = [
            # Ni減少系（他元素で置換）
            {"reduce": "Ni", "increase": ["Mn", "Co"], "ratios": [0.1, 0.2, 0.3]},
            # Co減少系（Ni rich化）
            {"reduce": "Co", "increase": ["Ni"], "ratios": [0.2, 0.5, 0.8]},
            # Mn減少系（Ni rich化）
            {"reduce": "Mn", "increase": ["Ni"], "ratios": [0.1, 0.2, 0.4]},
            # 二元素置換（Co+Mn減少 → Ni増加）
            {"reduce": ["Co", "Mn"], "increase": ["Ni"], "ratios": [0.2, 0.4]},
        ]
        
        for pattern in substitution_patterns:
            reduce_elements = pattern["reduce"] if isinstance(pattern["reduce"], list) else [pattern["reduce"]]
            increase_elements = pattern["increase"] if isinstance(pattern["increase"], list) else [pattern["increase"]]
            ratios = pattern["ratios"]
            
            for ratio in ratios:
                variant_atoms = self._create_substituted_structure(
                    base_atoms, reduce_elements, increase_elements, ratio, base_name
                )
                
                if variant_atoms is not None:
                    # 組成名作成
                    reduce_str = "_".join(reduce_elements)
                    increase_str = "_".join(increase_elements)
                    variant_name = f"{base_name}_{reduce_str}to{increase_str}_{int(ratio*100)}"
                    
                    # 構造最適化
                    variant_atoms = self.optimize_structure_systematic(variant_atoms, variant_name)
                    variants[variant_name] = variant_atoms
                    
                    # 組成分析
                    composition = self._analyze_composition(variant_atoms)
                    print(f"作成: {variant_name}")
                    print(f"  組成: {composition}")
        
        return variants
    
    def _create_substituted_structure(self, base_atoms, reduce_elements, increase_elements, ratio, base_name):
        """化学量論保持置換構造の作成"""
        variant_atoms = base_atoms.copy()
        
        # 置換対象原子の特定
        reduce_indices = []
        for element in reduce_elements:
            element_indices = [i for i, atom in enumerate(variant_atoms) if atom.symbol == element]
            reduce_indices.extend(element_indices)
        
        if not reduce_indices:
            print(f"警告: 減少対象元素 {reduce_elements} が見つかりません")
            return None
        
        # 増加対象原子の特定
        increase_indices = []
        for element in increase_elements:
            element_indices = [i for i, atom in enumerate(variant_atoms) if atom.symbol == element]
            increase_indices.extend(element_indices)
        
        # 置換数の計算
        total_reduce = len(reduce_indices)
        substitute_count = min(int(total_reduce * ratio), total_reduce)
        
        if substitute_count == 0:
            return None
        
        # ランダム選択（再現性確保）
        random.seed(42)
        selected_reduce = random.sample(reduce_indices, substitute_count)
        
        # 置換実行
        if len(increase_elements) == 1:
            # 単一元素への置換
            new_symbol = increase_elements[0]
            for idx in selected_reduce:
                variant_atoms[idx].symbol = new_symbol
        else:
            # 複数元素への分散置換
            for i, idx in enumerate(selected_reduce):
                new_symbol = increase_elements[i % len(increase_elements)]
                variant_atoms[idx].symbol = new_symbol
        
        return variant_atoms
    
    def _analyze_composition(self, atoms):
        """組成分析"""
        from collections import Counter
        symbols = [atom.symbol for atom in atoms]
        composition = Counter(symbols)
        
        # 遷移金属のみの比率計算
        tm_elements = ['Ni', 'Mn', 'Co']
        tm_counts = {elem: composition.get(elem, 0) for elem in tm_elements}
        total_tm = sum(tm_counts.values())
        
        if total_tm > 0:
            tm_ratios = {elem: count/total_tm for elem, count in tm_counts.items()}
            return f"Li{composition.get('Li', 0)}[Ni{tm_ratios['Ni']:.2f}Mn{tm_ratios['Mn']:.2f}Co{tm_ratios['Co']:.2f}]O{composition.get('O', 0)}"
        else:
            return str(dict(composition))
    
    def create_realistic_degradation_variants(self, base_atoms, base_name="NMC"):
        """実際の劣化パターンに基づく組成変動"""
        print(f"=== {base_name}の劣化パターン構造作成 ===")
        
        variants = {}
        
        # 実際の劣化パターン
        degradation_patterns = [
            # 表面Ni還元 (Ni2+ → Ni0, 表面でのNi損失)
            {"name": "surface_Ni_reduction", "reduce": "Ni", "increase": ["Mn"], "ratio": 0.1},
            
            # Co溶出 (電解液中への溶出)
            {"name": "Co_dissolution", "reduce": "Co", "increase": ["Ni"], "ratio": 0.3},
            
            # Mn溶出 (不均化反応によるMn損失)  
            {"name": "Mn_dissolution", "reduce": "Mn", "increase": ["Ni"], "ratio": 0.2},
            
            # リサイクル過程での選択的除去
            {"name": "selective_Co_removal", "reduce": "Co", "increase": ["Ni", "Mn"], "ratio": 0.7},
            
            # 高温処理によるCo/Mn損失
            {"name": "thermal_Co_Mn_loss", "reduce": ["Co", "Mn"], "increase": ["Ni"], "ratio": 0.3},
            
            # 極端Ni rich化 (NMC111 → NMC955相当)
            {"name": "extreme_Ni_rich", "reduce": ["Co", "Mn"], "increase": ["Ni"], "ratio": 0.8},
        ]
        
        for pattern in degradation_patterns:
            reduce_elements = pattern["reduce"] if isinstance(pattern["reduce"], list) else [pattern["reduce"]]
            increase_elements = pattern["increase"] if isinstance(pattern["increase"], list) else [pattern["increase"]]
            ratio = pattern["ratio"]
            name = pattern["name"]
            
            variant_atoms = self._create_substituted_structure(
                base_atoms, reduce_elements, increase_elements, ratio, base_name
            )
            
            if variant_atoms is not None:
                variant_name = f"{base_name}_{name}"
                
                # 構造最適化
                variant_atoms = self.optimize_structure_systematic(variant_atoms, variant_name)
                variants[variant_name] = variant_atoms
                
                # 組成分析
                composition = self._analyze_composition(variant_atoms)
                print(f"作成: {variant_name}")
                print(f"  組成: {composition}")
                print(f"  想定劣化: {name}")
        
        return variants
    
    def adjust_slab_size(self, slab, target_area=100.0):
        """スラブサイズの調整（格子不整合最小化）"""
        current_area = np.linalg.norm(np.cross(slab.cell[0], slab.cell[1]))
        scale_factor = np.sqrt(target_area / current_area)
        
        rep_x = max(1, int(scale_factor))
        rep_y = max(1, int(scale_factor))
        
        return slab.repeat([rep_x, rep_y, 1])
    
    def create_interface(self, slab1, slab2, separation=2.0, name="interface"):
        """界面構造作成"""
        print(f"界面作成: {name}, 分離距離: {separation} Å")
        
        # サイズ調整（格子不整合の最小化）
        area1 = np.linalg.norm(np.cross(slab1.cell[0], slab1.cell[1]))
        area2 = np.linalg.norm(np.cross(slab2.cell[0], slab2.cell[1]))
        target_area = max(area1, area2)
        
        slab1_adj = self.adjust_slab_size(slab1, target_area)
        slab2_adj = self.adjust_slab_size(slab2, target_area)
        
        # 界面作成
        interface = stack(slab1_adj, slab2_adj, axis=2, distance=separation)
        
        # 制約設定（両端を固定）
        z_coords = interface.positions[:, 2]
        z_min, z_max = z_coords.min(), z_coords.max()
        z_range = z_max - z_min
        
        # 下部20%と上部20%を固定
        bottom_threshold = z_min + z_range * 0.2
        top_threshold = z_max - z_range * 0.2
        
        fixed_indices = [i for i, z in enumerate(z_coords) 
                        if z <= bottom_threshold or z >= top_threshold]
        
        if fixed_indices:
            constraint = FixAtoms(indices=fixed_indices)
            interface.set_constraint(constraint)
        
        # 構造最適化
        interface = self.optimize_structure_systematic(interface, name)
        
        return interface
    
    def build_al_structures(self):
        """Al構造の構築"""
        print("=== Al構造構築 ===")
        
        # Al bulk構造作成
        al_bulk = ase_bulk('Al', 'fcc', a=4.046)
        al_bulk = self.optimize_structure_systematic(al_bulk, "Al_bulk")
        
        # Al表面構築
        al_surfaces = {}
        for miller in self.miller_indices[:3]:  # Al主要面のみ
            surface_name = f"Al_{miller[0]}{miller[1]}{miller[2]}"
            al_slab = self.makesurface_advanced(al_bulk, miller_indices=miller, 
                                              name=surface_name)
            al_surfaces[surface_name] = al_slab
            
        return al_bulk, al_surfaces
    
    def build_all_structures(self, cif_files):
        """全構造の体系的構築"""
        print("=" * 50)
        print("NMC-Al界面構造体系的構築開始")
        print("=" * 50)
        
        all_structures = {}
        all_surfaces = {}
        all_interfaces = {}
        
        # 1. Al構造構築
        al_bulk, al_surfaces = self.build_al_structures()
        all_structures["Al_bulk"] = al_bulk
        all_surfaces.update(al_surfaces)
        
        # 2. NMC系構造構築
        for material_name, cif_path in cif_files.items():
            if not os.path.exists(cif_path):
                print(f"警告: {cif_path}が見つかりません")
                continue
                
            print(f"\n=== {material_name}構造構築 ===")
            
            # 基本構造読み込み・最適化
            base_atoms = read(cif_path)
            base_atoms = self.optimize_structure_systematic(base_atoms, material_name)
            all_structures[material_name] = base_atoms
            
            # 組成変動構造作成（NMC系のみ）
            if "NMC" in material_name or "Li" in material_name:
                # 基本的な組成変動
                variants = self.create_composition_variants(base_atoms, material_name)
                
                # 実際の劣化パターンも追加
                degradation_variants = self.create_realistic_degradation_variants(base_atoms, material_name)
                variants.update(degradation_variants)
                
                all_structures.update(variants)
            else:
                variants = {material_name: base_atoms}
            
            # 各材料・組成の表面構築
            for variant_name, atoms in variants.items():
                for miller in self.miller_indices:
                    try:
                        surface_name = f"{variant_name}_{miller[0]}{miller[1]}{miller[2]}"
                        slab = self.makesurface_advanced(atoms, miller_indices=miller,
                                                       name=surface_name)
                        all_surfaces[surface_name] = slab
                    except Exception as e:
                        print(f"表面作成失敗: {surface_name}, エラー: {e}")
        
        # 3. 界面構造構築
        print("\n=== 界面構造構築 ===")
        al_reference = al_surfaces["Al_111"]  # Al(111)を基準面とする
        
        for surf_name, nmc_slab in all_surfaces.items():
            if surf_name.startswith("Al_"):
                continue
                
            try:
                interface_name = f"Al111_{surf_name}"
                interface = self.create_interface(al_reference, nmc_slab, 
                                                name=interface_name)
                all_interfaces[interface_name] = interface
            except Exception as e:
                print(f"界面作成失敗: {interface_name}, エラー: {e}")
        
        return all_structures, all_surfaces, all_interfaces
    
    def save_all_structures(self, structures, surfaces, interfaces):
        """全構造の保存"""
        print("\n=== 構造ファイル保存 ===")
        
        # ディレクトリ作成
        dirs = ["bulk", "surfaces", "interfaces"]
        for d in dirs:
            os.makedirs(os.path.join(self.output_dir, d), exist_ok=True)
        
        # バルク構造保存
        for name, atoms in structures.items():
            filepath = os.path.join(self.output_dir, "bulk", f"{name}.traj")
            write(filepath, atoms)
            print(f"保存: {filepath}")
        
        # 表面構造保存
        for name, atoms in surfaces.items():
            filepath = os.path.join(self.output_dir, "surfaces", f"{name}.traj")
            write(filepath, atoms)
        
        # 界面構造保存
        for name, atoms in interfaces.items():
            filepath = os.path.join(self.output_dir, "interfaces", f"{name}.traj")
            write(filepath, atoms)
        
        print(f"総保存数: バルク{len(structures)}, 表面{len(surfaces)}, 界面{len(interfaces)}")
    
    def generate_analysis_summary(self, structures, surfaces, interfaces):
        """解析サマリー生成"""
        summary = []
        summary.append("=" * 60)
        summary.append("NMC-Al界面構造構築サマリー")
        summary.append("=" * 60)
        
        summary.append(f"\n構築した構造数:")
        summary.append(f"  - バルク構造: {len(structures)}")
        summary.append(f"  - 表面構造: {len(surfaces)}")
        summary.append(f"  - 界面構造: {len(interfaces)}")
        
        summary.append(f"\n主要材料:")
        composition_types = {"pristine": 0, "substitution": 0, "degradation": 0}
        for name in structures.keys():
            if "pristine" in name:
                summary.append(f"  - {name} (純粋組成)")
                composition_types["pristine"] += 1
            elif "to" in name and any(x in name for x in ["Ni", "Co", "Mn"]):
                summary.append(f"  - {name} (元素置換)")
                composition_types["substitution"] += 1
            elif any(x in name for x in ["reduction", "dissolution", "loss", "rich"]):
                summary.append(f"  - {name} (劣化パターン)")
                composition_types["degradation"] += 1
            else:
                summary.append(f"  - {name}")
        
        summary.append(f"\n組成変動タイプ:")
        summary.append(f"  - 純粋組成: {composition_types['pristine']}")
        summary.append(f"  - 元素置換系: {composition_types['substitution']}")
        summary.append(f"  - 劣化パターン系: {composition_types['degradation']}")
        
        summary.append(f"\n検討ミラー指数 (NMC最頻面優先):")
        miller_descriptions = {
            (0, 0, 1): "最安定面、プレート形状",
            (1, 0, 4): "中活性、多面体形状", 
            (0, 1, 2): "高活性、切頭八面体形状",
            (0, 1, 0): "Li拡散活性面",
            (1, 1, 1): "Al最安定面",
            (1, 0, 0): "基本面",
            (1, 1, 0): "副次面"
        }
        for miller in self.miller_indices:
            desc = miller_descriptions.get(miller, "")
            summary.append(f"  - {miller} : {desc}")
        
        summary.append(f"\n化学量論保持置換の利点:")
        summary.append("  - 実際の劣化・リサイクルプロセスを反映")
        summary.append("  - 構造安定性の維持")
        summary.append("  - 界面特性の系統的評価が可能")
        
        summary.append(f"\n次のステップ:")
        summary.append("  1. カレンダリング圧力シミュレーション (10-200 MPa)")
        summary.append("  2. 熱処理プロセスシミュレーション (300-600°C)")
        summary.append("  3. 引張試験シミュレーション")
        summary.append("  4. 組成別分離効率評価")
        summary.append("  5. 最適リサイクル条件の決定")
        
        summary_text = "\n".join(summary)
        
        # ファイル保存
        with open(os.path.join(self.output_dir, "analysis_summary.txt"), "w") as f:
            f.write(summary_text)
        
        print(summary_text)
        return summary_text

def main():
    """メイン実行関数"""
    
    # ビルダー初期化
    builder = NMCInterfaceBuilder()
    
    # CIFファイル設定
    cif_files = {
        "NMC111": "/home/jovyan/Kaori/MD/input/Li3MnCoNiO6.cif",
        # 以下は利用可能な場合に追加
        "Li2O": "/home/jovyan/Kaori/MD/input/Li2O.cif",
        "CoO": "/home/jovyan/Kaori/MD/input/CoO.cif", 
        "MnO": "/home/jovyan/Kaori/MD/input/MnO.cif",
        "NiO": "/home/jovyan/Kaori/MD/input/NiO.cif",
    }
    
    # 利用可能なCIFファイルのみフィルター
    available_cifs = {name: path for name, path in cif_files.items() 
                     if os.path.exists(path)}
    
    if not available_cifs:
        print("エラー: 利用可能なCIFファイルがありません")
        return
    
    print(f"利用可能なCIFファイル: {list(available_cifs.keys())}")
    
    # 全構造構築
    structures, surfaces, interfaces = builder.build_all_structures(available_cifs)
    
    # 結果保存
    builder.save_all_structures(structures, surfaces, interfaces)
    
    # サマリー生成
    builder.generate_analysis_summary(structures, surfaces, interfaces)
    
    print("\n構築完了！次はカレンダリングシミュレーションに進んでください。")

if __name__ == "__main__":
    main()

KeyboardInterrupt: 

✅ 表面スラブの保存先: 'MD_Workflow_Output/surfaces'
🔍 '/home/jovyan/Kaori/MD/LiB_2/structure/output' から 74 個の最適化済みバルク構造を発見しました。

--- バルク 'NMC111_selective_Co_removal' から表面を生成中 ---
  -> ⚙️  (1, 1, 1) 面を生成中...
  -> ❌ エラー: (1, 1, 1) 面の生成に失敗しました。詳細: No EMT-potential for Li
  -> ⚙️  (0, 0, 1) 面を生成中...
  -> ❌ エラー: (0, 0, 1) 面の生成に失敗しました。詳細: 'EMT' object has no attribute 'nl'
  -> ⚙️  (1, 0, 4) 面を生成中...
  -> ❌ エラー: (1, 0, 4) 面の生成に失敗しました。詳細: 'EMT' object has no attribute 'nl'

--- バルク 'NMC111_extreme_Ni_rich' から表面を生成中 ---
  -> ⚙️  (1, 1, 1) 面を生成中...
  -> ❌ エラー: (1, 1, 1) 面の生成に失敗しました。詳細: No EMT-potential for Li
  -> ⚙️  (0, 0, 1) 面を生成中...
  -> ❌ エラー: (0, 0, 1) 面の生成に失敗しました。詳細: 'EMT' object has no attribute 'nl'
  -> ⚙️  (1, 0, 4) 面を生成中...
  -> ❌ エラー: (1, 0, 4) 面の生成に失敗しました。詳細: 'EMT' object has no attribute 'nl'

--- バルク 'NMC111_CotoNi_80_110_(1, 1, 0)' から表面を生成中 ---
  -> ⚙️  (1, 1, 1) 面を生成中...
  -> ❌ エラー: (1, 1, 1) 面の生成に失敗しました。詳細: No EMT-potential for Li
  -> ⚙️  (0, 0, 1) 面を生成中...
  -> ❌ エラー: (0, 0, 1) 面の生

UnknownFileTypeError: Empty file: /home/jovyan/Kaori/MD/LiB_2/structure/output/AlF3_0001_(0, 0, 1)_pos_opt.traj

In [ ]:
#!/usr/bin/env python3
"""
Trajectoryファイルの整理と、それに基づく界面構築・最適化を自動化するプログラム。

STEP 1: Trajectoryファイルの分析とクリーンアップ
  - 0KBの.trajファイル（失敗した計算）を削除
  - 正常な.trajファイルを初期/最終構造の.xyzに変換し、元ファイルを削除

STEP 2: 界面の構築と最適化
  - STEP 1で生成された最終構造(.xyz)を用いて界面を構築
  - Matlantisを用いて原子位置のみを最適化
"""
import os
import itertools
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display, clear_output
import nglview as nv

# ASE
from ase.io import read, write
from ase.build import stack

# Matlantis / PFP
from matlantis_features.atoms import MatlantisAtoms
from matlantis_features.features.common.opt import FireLBFGSASEOptFeature
from matlantis_features.utils.calculators import pfp_estimator_fn
from pfp_api_client.pfp.estimator import EstimatorCalcMode

# --- STEP 1: Trajectoryファイルの整理クラス ---
class TrajectoryCleanup:
    """
    .trajファイルを分析し、.xyzに変換してクリーンアップするクラス。
    """
    def __init__(self, directory):
        self.directory = Path(directory)
        print("--- STEP 1: Trajectoryファイルの分析と整理を開始 ---")
        if not self.directory.is_dir():
            print(f"🛑 エラー: ディレクトリ '{self.directory}' が見つかりません。")
            self.files_to_process = []
        else:
            self.files_to_process = list(self.directory.glob("*.traj"))
            print(f"🔍 '{self.directory}' から {len(self.files_to_process)} 個の .traj ファイルを発見。")

    def process_file(self, traj_path):
        """1つの.trajファイルを処理する。"""
        # ファイルサイズを確認
        try:
            file_size = traj_path.stat().st_size
        except FileNotFoundError:
            return # ファイルが存在しない場合はスキップ

        # 0KBのファイルは削除
        if file_size == 0:
            print(f"  -> 🗑️  0KBファイルを削除します: {traj_path.name}")
            os.remove(traj_path)
            return

        # 正常なファイルは変換
        base_name = traj_path.stem
        initial_xyz_path = self.directory / f"{base_name}_initial.xyz"
        final_xyz_path = self.directory / f"{base_name}_final.xyz"
        
        print(f"  -> 🔄 変換中: {traj_path.name}")
        try:
            # Trajectoryから初期構造と最終構造を読み込み
            atoms_list = read(traj_path, index=":")
            if not atoms_list:
                print(f"  -> ❌ 読み込みエラー（空のTrajectory）: {traj_path.name}")
                os.remove(traj_path)
                return

            # .xyzファイルとして保存
            write(str(initial_xyz_path), atoms_list[0])
            write(str(final_xyz_path), atoms_list[-1])
            
            # 元の.trajファイルを削除
            os.remove(traj_path)
            print(f"  -> ✔️ 変換成功、元のファイルを削除しました: {traj_path.name}")

        except Exception as e:
            print(f"  -> ❌ 変換中にエラーが発生 ({traj_path.name}): {e}")

    def run(self):
        """全ての対象ファイルに対して処理を実行する。"""
        if not self.files_to_process:
            print(" -> 処理対象の.trajファイルがないため、スキップします。")
            return
        for traj_file in self.files_to_process:
            self.process_file(traj_file)
        print("\n✨ STEP 1 完了。")


# --- Matlantis最適化関数 (共通) ---
def run_matlantis_optimization(atoms, trajectory_path, fmax=0.05, name="structure"):
    print(f"  -> Matlantis最適化開始 ({name}, fmax = {fmax}) ...")
    matlantis_atoms = MatlantisAtoms(atoms)
    estimator_function = pfp_estimator_fn(
        model_version='v7.0.0', calc_mode=EstimatorCalcMode.CRYSTAL_PLUS_D3
    )
    position_optimizer = FireLBFGSASEOptFeature(
        estimator_fn=estimator_function, filter=False, trajectory=str(trajectory_path),
        n_run=5000, fmax=fmax, show_progress_bar=True
    )
    try:
        result = position_optimizer(matlantis_atoms)
        optimized_atoms = result["atoms"]
        final_energy = optimized_atoms.get_potential_energy()
        print(f"  -> ✔️ 最適化完了！ エネルギー: {final_energy:.3f} eV")
        atoms.set_positions(optimized_atoms.get_positions())
        return True
    except Exception as e:
        print(f"  -> ❌ 最適化中にエラーが発生: {e}")
        return False

# --- STEP 2: 界面最適化クラス ---
class InterfaceOptimizer:
    def __init__(self, surfaces_dir, interfaces_dir):
        self.surfaces_dir = Path(surfaces_dir)
        self.interfaces_dir = Path(interfaces_dir)
        self.interfaces_dir.mkdir(parents=True, exist_ok=True)
        print(f"\n--- STEP 2: 界面構築・最適化プロセスを開始 ---")
        print(f"✅ 界面構造の保存先: '{self.interfaces_dir}'")

    def find_and_categorize_surfaces(self):
        print(f"🔍 '{self.surfaces_dir}' から最適化済み表面 (*_final.xyz) を分類中...")
        if not self.surfaces_dir.is_dir():
             print(f"🛑 エラー: 表面ディレクトリ '{self.surfaces_dir}' が見つかりません。")
             return [], []
        
        # 最終構造の.xyzファイルのみを対象にする
        all_surfaces = list(self.surfaces_dir.glob("*_final.xyz"))
        
        al_surfaces = [p for p in all_surfaces if "Al_" in p.stem]
        cathode_surfaces = [p for p in all_surfaces if "Al_" not in p.stem]
        print(f"  - Al表面: {len(al_surfaces)}個,  正極材表面: {len(cathode_surfaces)}個")
        return al_surfaces, cathode_surfaces

    def create_and_optimize_interface(self, slab1_path, slab2_path):
        # ファイル名から不要な部分を削除して命名
        slab1_name = slab1_path.stem.replace('_pos_opt_final', '').replace('_cell_opt_final', '')
        slab2_name = slab2_path.stem.replace('_pos_opt_final', '').replace('_cell_opt_final', '')
        interface_name = f"Interface_{slab1_name}_on_{slab2_name}"
        output_path = self.interfaces_dir / f"{interface_name}.cif"
        
        if output_path.exists():
            print(f"✅ 既存ファイルあり、スキップ: {output_path.name}")
            return

        print(f"\n--- 界面を構築・最適化: {slab1_name} + {slab2_name} ---")
        try:
            slab1, slab2 = read(slab1_path), read(slab2_path)
            interface = stack(slab1, slab2, axis=2, distance=2.5)
            interface.pbc = (True, True, False)
            interface.center(vacuum=10.0, axis=2)
            
            traj_path = self.interfaces_dir / f"{interface_name}.traj"
            if run_matlantis_optimization(interface, traj_path, fmax=0.05, name=interface_name):
                write(str(output_path), interface)
                print(f"  -> 💾 保存しました: {output_path.name}")
        except Exception as e:
            print(f"  -> ❌ エラー: {e}")

    def run(self):
        al_slabs, cathode_slabs = self.find_and_categorize_surfaces()
        if not al_slabs and not cathode_slabs:
            print(" -> 処理対象の表面ファイルがないため、終了します。")
            return
            
        print("\n--- カテゴリ1: Alを含む界面 ---")
        if al_slabs and cathode_slabs:
            for al_path in al_slabs:
                for cathode_path in cathode_slabs:
                    self.create_and_optimize_interface(al_path, cathode_path)
        else:
            print(" -> スキップ (Al表面または正極材表面のいずれかが不足)")
            
        print("\n--- カテゴリ2: Alを含まない界面 ---")
        if len(cathode_slabs) >= 2:
            for pair in itertools.combinations(cathode_slabs, 2):
                self.create_and_optimize_interface(pair[0], pair[1])
        else:
            print(" -> スキップ (正極材表面が2つ未満)")
        print("\n✨ STEP 2 完了。")

def show_results_viewer(results_dir):
    print(f"\n--- 🔬 結果ビューワー: '{results_dir}' ---")
    files = sorted(list(Path(results_dir).glob("*.cif")))
    if not files:
        print("表示できる構造がありません。")
        return
    options = {"-- 選択してください --": None}
    options.update({p.name: p for p in files})
    widgets.interact(
        lambda file_path: display(nv.show_ase(read(file_path)) if file_path else None),
        file_path=widgets.Dropdown(options=options, description='Interface:', layout={'width': 'max-content'})
    )

# --- メイン実行部 ---
if __name__ == "__main__":
    # ===== 設定項目 =====
    # 1. Trajectoryファイルが保存されているディレクトリ
    OPTIMIZATION_OUTPUT_DIR = "/home/jovyan/Kaori/MD/LiB_2/structure/output"
    
    # 2. 最適化した界面の保存先
    INTERFACES_DIR = "/home/jovyan/Kaori/MD/LiB_2/structure/output/optimized_interfaces")
    # ====================

    clear_output(wait=True)
    
    # STEP 1: メモリ管理 (traj -> xyz変換 & 削除)
    cleanup = TrajectoryCleanup(OPTIMIZATION_OUTPUT_DIR)
    cleanup.run()
    
    # STEP 2: 界面の構築と最適化
    # 表面ファイルも同じディレクトリにあるため、surfaces_dirとして同じパスを指定
    interface_opt = InterfaceOptimizer(surfaces_dir=OPTIMIZATION_OUTPUT_DIR, interfaces_dir=INTERFACES_DIR)
    interface_opt.run()
    
    print("\n\n✨ 全てのワークフローが完了しました。 ✨")
    
    # STEP 3: 結果の表示
    show_results_viewer(INTERFACES_DIR)


In [25]:
from ase.build import standardize_cell

ImportError: cannot import name 'standardize_cell' from 'ase.build' (/home/jovyan/.py311/lib/python3.11/site-packages/ase/build/__init__.py)